# Installing Libraries

In [2]:
%%sh

sudo su 

#Download appropriate package for the OS version
#Choose only ONE of the following, corresponding to your OS version

#RHEL 7 and Oracle Linux 7
curl https://packages.microsoft.com/config/rhel/7/prod.repo > /etc/yum.repos.d/mssql-release.repo

#RHEL 8 and Oracle Linux 8
#curl https://packages.microsoft.com/config/rhel/8/prod.repo > /etc/yum.repos.d/mssql-release.repo

#RHEL 9
#curl https://packages.microsoft.com/config/rhel/9.0/prod.repo > /etc/yum.repos.d/mssql-release.repo

exit

sudo yum remove unixODBC-utf16 unixODBC-utf16-devel #to avoid conflicts
sudo ACCEPT_EULA=Y yum install -y msodbcsql17
# optional: for bcp and sqlcmd
sudo ACCEPT_EULA=Y yum install -y mssql-tools
echo 'export PATH="$PATH:/opt/mssql-tools/bin"' >> ~/.bashrc
source ~/.bashrc
# optional: for unixODBC development headers
sudo yum install -y unixODBC-devel

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   413  100   413    0     0   7736      0 --:--:-- --:--:-- --:--:--  7792


Loaded plugins: dkms-build-requires, extras_suggestions, kernel-livepatch,
              : langpacks, priorities, update-motd, versionlock


No Match for argument: unixODBC-utf16
No Match for argument: unixODBC-utf16-devel


No Packages marked for removal
Loaded plugins: dkms-build-requires, extras_suggestions, kernel-livepatch,
              : langpacks, priorities, update-motd, versionlock


https://download.docker.com/linux/centos/2/x86_64/stable/repodata/repomd.xml: [Errno 14] HTTPS Error 404 - Not Found
Trying other mirror.


67 packages excluded due to repository priority protections
Package msodbcsql17-17.10.6.1-1.x86_64 already installed and latest version
Nothing to do
Loaded plugins: dkms-build-requires, extras_suggestions, kernel-livepatch,
              : langpacks, priorities, update-motd, versionlock


https://download.docker.com/linux/centos/2/x86_64/stable/repodata/repomd.xml: [Errno 14] HTTPS Error 404 - Not Found
Trying other mirror.


67 packages excluded due to repository priority protections
Package mssql-tools-17.10.1.1-1.x86_64 already installed and latest version
Nothing to do
Loaded plugins: dkms-build-requires, extras_suggestions, kernel-livepatch,
              : langpacks, priorities, update-motd, versionlock


https://download.docker.com/linux/centos/2/x86_64/stable/repodata/repomd.xml: [Errno 14] HTTPS Error 404 - Not Found
Trying other mirror.


67 packages excluded due to repository priority protections
Package unixODBC-devel-2.3.1-15.amzn2.x86_64 already installed and latest version
Nothing to do


In [3]:
!conda install polars -y -q

Channels:
 - conda-forge
 - nvidia
 - pytorch
Platform: linux-64
Solving environment: ...working... done

# All requested packages already installed.



In [4]:
!pip install icd-mappings pyreadr -q

# Importing libraries

In [5]:
import pyodbc
import requests
import re
import pyreadr

In [6]:
import polars as pl
import numpy as np
import time
import pickle
import matplotlib.pyplot as plt

# Connecting to database

In [7]:
file_path = '/home/ec2-user/SageMaker/db-credentials.txt'

with open(file_path, 'r') as file:
    # Code to parse the data will go here
    file_contents = file.read()

# Remove newlines and extra spaces
cleaned_string = file_contents.replace('\n', '').strip()

# Extract variable-value pairs using regular expressions
pattern = r'"([^"]+)": "([^"]+)"'
pairs = re.findall(pattern, cleaned_string)

parsed_data = {}

for variable, data in pairs:
    parsed_data[variable] = data

In [8]:
### This example shows how to view tables in the AAOCHIN2023 database. 
raw_db = 'AAOCHIN2023' 
raw_connection_string = "DRIVER={ODBC Driver 17 for SQL Server};" + \
                    "SERVER=" + parsed_data['host'] + ',' + parsed_data['port'] + ';' + \
                    "DATABASE=" + raw_db + ';' + \
                    "UID=" + parsed_data['username'] + ';' + \
                    "PWD={" + parsed_data['password'] + "};"
raw_conn = pyodbc.connect(raw_connection_string, trusted_connection = 'no')
raw_cursor = raw_conn.cursor()

In [9]:
#### CHECK IF DATABASE CURSOR IS WORKING
raw_cursor.execute('''
SELECT name AS TABLE_NAME
FROM sys.views
''')
for row in raw_cursor:
    print(row[0])

OBSERVATION_FACT
PATIENT_DIMENSION
PROVIDER_DIMENSION
VISIT_DIMENSION
ZCTA_CVS
SIDECAR_FACILITIES
CONCEPT_DIMENSION
I2B2


# SQL queries for demographics, lab results, medications and diagnoses

In [9]:
###### CONFIG ######
class cohort_data_config:
    get_lab_results_all = True
    get_latest_lab_results =  False
    get_temporal_lab_results_only_mean = False
    get_temporal_lab_results_only_mean_latest_oldest = False

## Testing (NO NEED to run)

In [10]:
# test_df = pl.read_database("""  """,
#     connection=raw_cursor,
# ) 

In [ ]:
# test_df

# Cohort

In [12]:
cohort_df = pl.read_database(""" SELECT * from S348.dbo.DETERMINE_OCHIN_COHORT""",
    connection=raw_cursor,
) 

In [13]:
cohort_df.write_parquet("Determine_Cohort_with_labels_OutcomeDate.parquet")

In [14]:
cohort_df.head()

PATIENT_NUM,Index_Start_date,FirstOutcomeDate,Age_at_encounter_date,Outcome,DEATH_DATE,Zip_CD
i64,datetime[μs],datetime[μs],i64,i64,datetime[μs],str
8202921,2017-03-15 00:00:00,2019-05-29 00:00:00,60,1,null,"""83706"""
6046573,2017-03-27 00:00:00,null,45,0,null,"""97136"""
2179286,2016-12-13 00:00:00,null,34,0,null,"""77530"""
6861988,2016-09-16 00:00:00,null,33,0,null,"""90035"""
7452088,2017-02-02 00:00:00,null,43,0,null,"""75217"""


## Diagnoses

In [ ]:
diag_cohort_df = pl.read_database(
    query='''SELECT o.PATIENT_NUM, o.ENCOUNTER_NUM, o.START_DATE, o.CONCEPT_CD
FROM AAOCHIN2023.S348.OBSERVATION_FACT o
JOIN S348.dbo.DETERMINE_OCHIN_COHORT c ON o.PATIENT_NUM = c.PATIENT_NUM
WHERE o.PATIENT_NUM IN (SELECT PATIENT_NUM FROM S348.dbo.DETERMINE_OCHIN_COHORT)
  AND o.START_DATE >= DATEADD(year, -1, c.Index_Start_date)
  AND o.START_DATE < c.Index_Start_date
  AND o.Concept_CD NOT LIKE 'ICD10CM:E11%'
  AND o.Concept_CD NOT LIKE 'ICD9CM:250%'

  AND o.Concept_CD LIKE 'ICD%'
ORDER BY o.PATIENT_NUM;''',
    connection=raw_cursor,
)

In [ ]:
### saving diagnoses_df ###
diag_cohort_df.write_parquet("Determine_diagnoses_cohort_data.parquet")

In [ ]:
del diag_cohort_df

## Medications

In [ ]:
med_cohort_df = pl.read_database(
    query='''SELECT 
    o.PATIENT_NUM, 
    o.ENCOUNTER_NUM, 
    o.START_DATE, 
    o.CONCEPT_CD AS RxNorm, 
    c.NAME_CHAR AS Medication_Name,
    o.START_DATE as RX_StartDate, 
    o.RX_DOSE_ORDERED as RX_Dose_Ordered, 
    o.RX_DOSE_ORDERED_UNIT as Rx_Dose_Ordered_Unit, 
    o.RX_QUANTITY as RX_Quantity
FROM 
    AAOCHIN2023.S348.OBSERVATION_FACT o
JOIN 
    AAOCHIN2023.common.CONCEPT_DIMENSION c 
    ON o.CONCEPT_CD = c.CONCEPT_CD
JOIN 
    S348.dbo.DETERMINE_OCHIN_COHORT dc 
    ON o.PATIENT_NUM = dc.PATIENT_NUM
WHERE 
    o.PATIENT_NUM IN (SELECT PATIENT_NUM FROM S348.dbo.DETERMINE_OCHIN_COHORT)
    AND o.CONCEPT_CD LIKE 'RXNORM%' -- pattern for medication data
    AND o.START_DATE >= DATEADD(year, -1, dc.Index_Start_date)
    AND o.START_DATE <= dc.Index_Start_date
ORDER BY 
    o.PATIENT_NUM, RX_Quantity;''',
    connection=raw_cursor,
) 

In [ ]:
med_cohort_df.shape

In [ ]:
### saving medications_df ###
med_cohort_df.write_parquet('Determine_medications_cohort_data.parquet')

## Lab results

In [ ]:
if  cohort_data_config.get_lab_results_all:
    lab_results_df = pl.read_database(
    query='''WITH RankedResults AS (
    SELECT 
        o.PATIENT_NUM, 
        o.ENCOUNTER_NUM, 
        o.START_DATE as Specimen_Date,
        o.CONCEPT_CD AS LabLOINC, 
        c.NAME_CHAR AS Lab_Name,
        o.NVAL_NUM as Result_Number, 
        o.UNITS_CD as Result_Unit
        --ROW_NUMBER() OVER (PARTITION BY o.PATIENT_NUM, o.CONCEPT_CD ORDER BY o.START_DATE DESC) as rn
    FROM 
        AAOCHIN2023.S348.OBSERVATION_FACT o
    JOIN 
        AAOCHIN2023.common.CONCEPT_DIMENSION c ON o.CONCEPT_CD = c.CONCEPT_CD
    JOIN 
        S348.dbo.DETERMINE_OCHIN_COHORT coh ON o.PATIENT_NUM = coh.PATIENT_NUM
    WHERE
        o.PATIENT_NUM IN (SELECT PATIENT_NUM FROM S348.dbo.DETERMINE_OCHIN_COHORT)
        AND o.CONCEPT_CD LIKE 'LOINC%'
        AND o.CONCEPT_CD != 'LOINC:4548-4' -- Excluding A1c data
        AND o.CONCEPT_CD != 'LOINC:17856-6' -- Excluding A1c data
        AND o.CONCEPT_CD != 'LOINC:4549-2' -- Excluding A1c data
        AND o.START_DATE >= DATEADD(YEAR, -1, coh.Index_Start_date)
        AND o.START_DATE < coh.Index_Start_date
        AND o.NVAL_NUM IS NOT NULL 
    )
    SELECT 
        PATIENT_NUM,
        ENCOUNTER_NUM,
        Specimen_Date,
        LabLOINC,
        Lab_Name,
        Result_Number,
        Result_Unit

    FROM

        RankedResults''',
    connection=raw_cursor)
    
    lab_results_df.write_parquet('Determine_lab_results_cohort_data_all.parquet')
    
    



elif cohort_data_config.get_latest_lab_results:
    lab_results_df = pl.read_database(
    query='''WITH RankedResults AS (
    SELECT 
        o.PATIENT_NUM, 
        o.ENCOUNTER_NUM, 
        o.START_DATE as Specimen_Date,
        o.CONCEPT_CD AS LabLOINC, 
        c.NAME_CHAR AS Lab_Name,
        o.NVAL_NUM as Result_Number, 
        o.UNITS_CD as Result_Unit,
        ROW_NUMBER() OVER (PARTITION BY o.PATIENT_NUM, o.CONCEPT_CD ORDER BY o.START_DATE DESC) as rn
    FROM 
        AAOCHIN2023.S348.OBSERVATION_FACT o
    JOIN 
        AAOCHIN2023.common.CONCEPT_DIMENSION c ON o.CONCEPT_CD = c.CONCEPT_CD
    JOIN 
        S348.dbo.DETERMINE_OCHIN_COHORT coh ON o.PATIENT_NUM = coh.PATIENT_NUM
    WHERE
        o.PATIENT_NUM IN (SELECT PATIENT_NUM FROM S348.dbo.DETERMINE_OCHIN_COHORT)
        AND o.CONCEPT_CD LIKE 'LOINC%'
        AND o.CONCEPT_CD != 'LOINC:4548-4' -- Excluding A1c data
        AND o.CONCEPT_CD != 'LOINC:17856-6' -- Excluding A1c data
        AND o.CONCEPT_CD != 'LOINC:4549-2' -- Excluding A1c data
        AND o.START_DATE >= DATEADD(YEAR, -1, coh.Index_Start_date)
        AND o.START_DATE < coh.Index_Start_date
        AND o.NVAL_NUM IS NOT NULL 
)
SELECT 
    PATIENT_NUM,
    ENCOUNTER_NUM,
    Specimen_Date,
    LabLOINC,
    Lab_Name,
    Result_Number,
    Result_Unit
FROM 
    RankedResults
WHERE 
    rn = 1
ORDER BY 
    PATIENT_NUM;''',
    connection=raw_cursor)
    
    lab_results_df.write_parquet('Determine_lab_results_cohort_data_latest.parquet')
    
elif cohort_data_config.get_temporal_lab_results_only_mean:
    
    lab_results_df = pl.read_database(
    query='''WITH MeanLabResults AS (
    SELECT 
        o.PATIENT_NUM, 
        o.CONCEPT_CD AS LabLOINC, 
        AVG(o.NVAL_NUM) as Mean_Result_Number
        
    FROM 
        AAOCHIN2023.S348.OBSERVATION_FACT o
    JOIN 
        AAOCHIN2023.common.CONCEPT_DIMENSION c ON o.CONCEPT_CD = c.CONCEPT_CD
    JOIN 
        S348.dbo.DETERMINE_OCHIN_COHORT coh ON o.PATIENT_NUM = coh.PATIENT_NUM
    WHERE 
        o.CONCEPT_CD LIKE 'LOINC%'
        AND o.CONCEPT_CD != 'LOINC:4548-4' -- Excluding A1c data
        AND o.CONCEPT_CD != 'LOINC:17856-6' -- Excluding A1c data
        AND o.CONCEPT_CD != 'LOINC:4549-2' -- Excluding A1c data
        AND o.START_DATE >= DATEADD(YEAR, -1, coh.Index_Start_date)
        AND o.START_DATE < coh.Index_Start_date
        AND o.NVAL_NUM IS NOT NULL 
    GROUP BY 
        o.PATIENT_NUM, 
        o.CONCEPT_CD
)
SELECT 
    PATIENT_NUM,
    LabLOINC,
    Mean_Result_Number
    
FROM 
    MeanLabResults
ORDER BY 
    PATIENT_NUM;''',
    connection=raw_cursor)
    
    lab_results_df.write_parquet('Determine_lab_results_cohort_data_mean.parquet')
    
elif cohort_data_config.get_temporal_lab_results_only_mean_latest_oldest:
    lab_results_df = pl.read_database(
    query='''WITH LabResults AS (
    SELECT 
        o.PATIENT_NUM, 
        o.ENCOUNTER_NUM, 
        o.START_DATE AS Specimen_Date,
        o.CONCEPT_CD AS LabLOINC, 
        c.NAME_CHAR AS Lab_Name,
        o.NVAL_NUM AS Result_Number, 
        ROW_NUMBER() OVER (PARTITION BY o.PATIENT_NUM, o.CONCEPT_CD ORDER BY o.START_DATE DESC) AS rn_latest,
        ROW_NUMBER() OVER (PARTITION BY o.PATIENT_NUM, o.CONCEPT_CD ORDER BY o.START_DATE ASC) AS rn_oldest
    FROM 
        AAOCHIN2023.S348.OBSERVATION_FACT o
    JOIN 
        AAOCHIN2023.common.CONCEPT_DIMENSION c ON o.CONCEPT_CD = c.CONCEPT_CD
    JOIN 
        S348.dbo.DETERMINE_OCHIN_COHORT coh ON o.PATIENT_NUM = coh.PATIENT_NUM
    WHERE 
        o.CONCEPT_CD LIKE 'LOINC%'
        AND o.CONCEPT_CD != 'LOINC:4548-4' -- Excluding A1c data
        AND o.CONCEPT_CD != 'LOINC:17856-6' -- Excluding A1c data
        AND o.CONCEPT_CD != 'LOINC:4549-2' -- Excluding A1c data
        AND o.START_DATE >= DATEADD(YEAR, -1, coh.Index_Start_date)
        AND o.START_DATE < coh.Index_Start_date
        AND o.NVAL_NUM IS NOT NULL
),
AggregatedLabResults AS (
    SELECT
        PATIENT_NUM, 
        LabLOINC, 
        Lab_Name,
        Result_Unit,
        AVG(Result_Number) AS Mean_Result
    FROM 
        LabResults
    GROUP BY 
        PATIENT_NUM, LabLOINC, Lab_Name, Result_Unit
),
LatestAndOldestResults AS (
    SELECT 
        PATIENT_NUM, 
        LabLOINC, 
        Lab_Name,
        MAX(CASE WHEN rn_latest = 1 THEN Result_Number END) AS Latest_Result_Number,
        MAX(CASE WHEN rn_oldest = 1 THEN Result_Number END) AS Oldest_Result_Number,

    FROM 
        LabResults
    WHERE 
        rn_latest = 1 OR rn_oldest = 1
    GROUP BY 
        PATIENT_NUM, LabLOINC
)

SELECT 
    a.PATIENT_NUM,
    a.LabLOINC,
    a.Lab_Name,
    l.Latest_Result_Number,
    l.Oldest_Result_Number,
    a.Mean_Result,
FROM 
    AggregatedLabResults a
JOIN 
    LatestAndOldestResults l 
    ON a.PATIENT_NUM = l.PATIENT_NUM 
    AND a.LabLOINC = l.LabLOINC 
ORDER BY 
    a.PATIENT_NUM;''',
    connection=raw_cursor)
    
    lab_results_df.write_parquet('Determine_lab_results_cohort_data_latest_oldest_mean.parquet')
    

In [ ]:
lab_results_df.shape

## Demographics

In [ ]:
demo_cohort_df = pl.read_database(
    query='''SELECT pd.Patient_num, pd.Sex_CD, pd.Race_CD, pd.Hispanic_CD, pd.Gender_CD
FROM AAOCHIN2023.S348.PATIENT_DIMENSION pd
WHERE pd.PATIENT_NUM in (SELECT PATIENT_NUM FROM S348.dbo.DETERMINE_OCHIN_COHORT);''',
    connection=raw_cursor,
) 

In [ ]:
demo_cohort_df.write_parquet("Determine_demographics_cohort_data.parquet")

## BMI, HEIGHT, WEIGHT and BP

In [ ]:
bmi_cohort_df = pl.read_database(
    query=''' SELECT ob.PATIENT_NUM, ob.CONCEPT_CD, ob.NVAL_NUM, UNITS_CD, ob.START_DATE
    FROM 
        AAOCHIN2023.S348.OBSERVATION_FACT ob
    JOIN
        S348.dbo.DETERMINE_OCHIN_COHORT dc 
        ON ob.PATIENT_NUM = dc.PATIENT_NUM
    WHERE
        ob.PATIENT_NUM IN (SELECT PATIENT_NUM FROM S348.dbo.DETERMINE_OCHIN_COHORT)
        AND ob.START_DATE >= DATEADD(YEAR, -1, dc.Index_Start_date)
        AND ob.START_DATE < dc.Index_Start_date
        AND (ob.CONCEPT_CD LIKE 'VIT|BMI%'
        OR ob.CONCEPT_CD LIKE 'VIT|DIASTOLIC%'
        OR ob.CONCEPT_CD LIKE 'VIT|SYSTOLIC%'
        OR ob.CONCEPT_CD LIKE 'VIT|WT%'
        );''',
    connection=raw_cursor,
) 

In [ ]:
bmi_cohort_df.head()

In [ ]:
bmi_cohort_df.shape

In [ ]:
bmi_cohort_df.write_parquet("Determine_dbmi_bp_cohort_data.parquet")

## Height


In [ ]:
height_df = pl.read_database(""" SELECT ob.PATIENT_NUM, cd.CONCEPT_PATH, cd.NAME_CHAR, ob.NVAL_NUM, ob.UNITS_CD
    FROM 
        AAOCHIN2023.S348.OBSERVATION_FACT ob     
    JOIN
        AAOCHIN2023.common.CONCEPT_DIMENSION cd
        ON ob.CONCEPT_CD = cd.CONCEPT_CD
    WHERE
        ob.PATIENT_NUM IN (SELECT PATIENT_NUM FROM S348.dbo.DETERMINE_OCHIN_COHORT)
        AND cd.CONCEPT_PATH LIKE '\i2b2\Vitals\H%';  """,
    connection=raw_cursor,
) 

In [ ]:
height_df.head()

In [ ]:
height_df.write_parquet("Determine_height_cohort_data.parquet")

## SDoH datasets

In [ ]:
tob_cohort_df = pl.read_database(
    query='''SELECT ob.PATIENT_NUM, ob.CONCEPT_CD, ob.START_DATE
    FROM 
        AAOCHIN2023.S348.OBSERVATION_FACT ob
    JOIN
        S348.dbo.DETERMINE_OCHIN_COHORT dc 
        ON ob.PATIENT_NUM = dc.PATIENT_NUM
    WHERE
        ob.PATIENT_NUM IN (SELECT PATIENT_NUM FROM S348.dbo.DETERMINE_OCHIN_COHORT)
        AND ob.START_DATE >= DATEADD(YEAR, -1, dc.Index_Start_date)
        AND ob.START_DATE < dc.Index_Start_date
        AND (SUBSTRING(CONCEPT_CD, 1,7) = 'VIT|TOB'
        OR SUBSTRING(CONCEPT_CD, 1,7) = 'VIT|SMK');''',
    connection=raw_cursor,
) 

In [ ]:
len(np.unique(tob_cohort_df['PATIENT_NUM'].to_list()))

In [ ]:
tob_cohort_df.write_parquet('Determine_tob_cohort_data.parquet')

 ### SDOH screening data

In [33]:
sdoh_screening_df = pl.read_database(
    query=''' SELECT TOP 10 *
    FROM 
        AAOCHIN2023.S348.OBSERVATION_FACT
     ''',
    connection=raw_cursor,
) 
sdoh_screening_df.head()

ENCOUNTER_NUM,PATIENT_NUM,CONCEPT_CD,PROVIDER_ID,START_DATE,MODIFIER_CD,INSTANCE_NUM,VALTYPE_CD,TVAL_CHAR,NVAL_NUM,VALUEFLAG_CD,QUANTITY_NUM,UNITS_CD,END_DATE,LOCATION_CD,RX_START_DATE,RX_DOSE_ORDERED,RX_DOSE_ORDERED_UNIT,RX_DOSE_FORM,RX_REFILLS,RX_QUANTITY
i64,i64,str,str,datetime[μs],str,i64,str,str,"decimal[18,5]",str,"decimal[18,5]",str,datetime[μs],str,date,"decimal[15,5]",str,str,"decimal[15,8]","decimal[15,8]"
222552013,8683894,"""CPT4:0064U""","""6AECE123-1D16-4E22-85C3-827D98…",2021-12-22 00:00:00,"""@""",1,"""""","""""",null,"""""",null,"""@""",2021-12-22 00:00:00,"""DE4F5B35-FE32-4BC2-9CF8-30DF2D…",null,null,null,null,null,null
222012359,9309612,"""CPT4:0064U""","""105B8449-FF33-4435-A351-97AF14…",2021-12-15 00:00:00,"""@""",1,"""""","""""",null,"""""",null,"""@""",2021-12-15 00:00:00,"""DE4F5B35-FE32-4BC2-9CF8-30DF2D…",null,null,null,null,null,null
206904254,8097180,"""CPT4:0064U""","""105B8449-FF33-4435-A351-97AF14…",2021-08-18 00:00:00,"""@""",1,"""""","""""",null,"""""",null,"""@""",2021-08-18 00:00:00,"""DE4F5B35-FE32-4BC2-9CF8-30DF2D…",null,null,null,null,null,null
185878932,7957728,"""CPT4:0064U""","""E6DFD235-372F-4C28-B1A5-FD8653…",2021-02-04 00:00:00,"""@""",1,"""""","""""",null,"""""",null,"""@""",2021-02-04 00:00:00,"""DE4F5B35-FE32-4BC2-9CF8-30DF2D…",null,null,null,null,null,null
206273037,7966046,"""CPT4:0064U""","""6AECE123-1D16-4E22-85C3-827D98…",2021-08-13 00:00:00,"""@""",1,"""""","""""",null,"""""",null,"""@""",2021-08-13 00:00:00,"""DE4F5B35-FE32-4BC2-9CF8-30DF2D…",null,null,null,null,null,null


In [34]:
sdoh_screening_df = pl.read_database(
    query=''' SELECT ob.PATIENT_NUM, cd.CONCEPT_PATH, cd.NAME_CHAR, cd.CONCEPT_CD, ob.QUANTITY_NUM
    FROM 
        AAOCHIN2023.S348.OBSERVATION_FACT ob     
    JOIN
        AAOCHIN2023.common.CONCEPT_DIMENSION cd
        ON ob.CONCEPT_CD = cd.CONCEPT_CD
    JOIN
        S348.dbo.DETERMINE_OCHIN_COHORT dc 
        ON ob.PATIENT_NUM = dc.PATIENT_NUM
        
    WHERE
        ob.PATIENT_NUM IN (SELECT PATIENT_NUM FROM S348.dbo.DETERMINE_OCHIN_COHORT)
        AND cd.CONCEPT_PATH LIKE '\i2b2\SDH\%' 
        AND ob.START_DATE < dc.Index_Start_date''',
    connection=raw_cursor,
) 


In [35]:
sdoh_screening_df.shape

(88857, 5)

In [36]:
sdoh_screening_df.head()

PATIENT_NUM,CONCEPT_PATH,NAME_CHAR,CONCEPT_CD,QUANTITY_NUM
i64,str,str,str,"decimal[18,5]"
6833020,"""\i2b2\SDH\Food Insecurity\ADV0…","""Did your household run out of …","""SDH:ADV0041 """,0.00000
2240700,"""\i2b2\SDH\Food Insecurity\ADV0…","""Did your household run out of …","""SDH:ADV0041 """,0.00000
2790074,"""\i2b2\SDH\Food Insecurity\ADV0…","""Did your household run out of …","""SDH:ADV0041 """,0.00000
2116451,"""\i2b2\SDH\Food Insecurity\ADV0…","""Did your household run out of …","""SDH:ADV0041 """,0.00000
4102386,"""\i2b2\SDH\Food Insecurity\ADV0…","""Did your household run out of …","""SDH:ADV0041 """,0.00000


In [37]:
len(np.unique(sdoh_screening_df['PATIENT_NUM'].to_list()))

11447

In [38]:
sdoh_screening_df.write_parquet('Determine_Cohort_sdoh_screening_questions.parquet')

# Community Vital Signs

In [ ]:
community_vital_df = pl.read_database(
    query=''' SELECT TOP 20 *
    FROM 
        AAOCHIN2023.S348.ZCTA_CVS;''',
    connection=raw_cursor,
) 

In [ ]:
community_vital_df.head(15)

In [ ]:
community_vital_df = pl.read_database(
    query=''' SELECT pd.PATIENT_NUM, pd.ZIP_CD, cvs.Indicator, cvs.PERIOD, cvs.FACT
    FROM 
        AAOCHIN2023.S348.PATIENT_DIMENSION pd       
    JOIN
        AAOCHIN2023.S348.ZCTA_CVS cvs
        ON pd.ZIP_CD = cvs.ZCTA
    WHERE
        pd.PATIENT_NUM IN (SELECT PATIENT_NUM FROM S348.dbo.DETERMINE_OCHIN_COHORT)
        AND cvs.PERIOD = '2016-2020'
        AND (cvs.Indicator = 'ACS_GINI'
        OR cvs.Indicator = 'ACS_MedHHIncome'
        OR cvs.Indicator = 'ACS_pctCollGrad'
        OR cvs.Indicator = 'ACS_pctPoverty100'
        OR cvs.Indicator = 'ACS_Unemployment');''',
    connection=raw_cursor,
) 

In [ ]:
len(np.unique(community_vital_df['PATIENT_NUM'].to_list()))

In [ ]:
community_vital_df.write_parquet('Determine_community_vital_signs_cohort.parquet')

In [ ]:
print('Hello')

# Patients with no visits/encounters after index visit

In [10]:
pids_no_encs_after_vi_df = pl.read_database(
    query='''SELECT * FROM S348.dbo.patient_records_after_index_visit''',
    connection=raw_cursor,
) 

In [11]:
pids_no_encs_after_vi_df.write_parquet('Determine_cohort_after_visit_index_details.parquet')

# Helper functions and variables

In [9]:
def get_active_ingredient(rxcui):
       # Define the base URL of the RxNorm API
       base_url = "https://rxnav.nlm.nih.gov/REST/rxcui/"
       
       # Create the complete URL for the request
       request_url = f"{base_url}{rxcui}/related.json?tty=IN"
       
       try:
           # Make the GET request to the API
           response = requests.get(request_url)
           response.raise_for_status()  # Raise an error if the request was unsuccessful
           print(response)
           # Parse the JSON response
           data = response.json()
#            print(data)
           # Extract the active ingredient from the JSON response
           ingredients = data.get('relatedGroup', {}).get('conceptGroup', [])
           for group in ingredients:
               if group.get('tty') == 'IN':  # Here we're looking for ingredients
                   active_ingredients = [concept['name'] for concept in group.get('conceptProperties', [])]
                   if active_ingredients == []:
                    print(data)
                   return active_ingredients
       
       except requests.RequestException as e:
           print(f"An error occurred: {e}")

       try:
            response = requests.url(f'https://rxnav.nlm.nih.gov/REST/rxcui/{rxcui}/historystatus.json?caller=RxNav')
            data = response.json()
                        
            active_ingredient_name = data['rxcuiStatusHistory']['definitionalFeatures'
                                                                   ]['ingredientAndStrength'
                                                                    ][0]['activeIngredientName']
            return active_ingredient_name
                
       except:
            return []
            
def get_drug_class(rxcui):
       # Endpoint to get the ATC classification
       url = f"https://rxnav.nlm.nih.gov/REST/rxcui/{rxcui}/property.json?propName=ATC"
       
       # Make the HTTP GET request
       response = requests.get(url)
       
       # Check if the request was successful
       if response.status_code == 200:
           data = response.json()
           try: 
               unique_class_names = {item['rxclassMinConceptItem']['className'] for item in data['rxclassDrugInfoList']['rxclassDrugInfo']}
           except:
                return "No data"
       else:
           return f"Failed to retrieve data: {response.status_code}"

In [10]:
### loading domain expert lab result features
with open('loincs_hem_chem.pkl', 'rb') as file:
    hem_loics  = pickle.load(file)

In [11]:
### ladoing phe codes (ICD10 -> phe code) and (ICD9 -> phecode  )

 # EDA & Pre-Processing
#### Data dict:  https://view.officeapps.live.com/op/view.aspx?src=https%3A%2F%2Fwww.aim-ahead.net%2Fmedia%2Fhn1f2blb%2Fochindatadictionary_year2.xlsx&wdOrigin=BROWSELINK

#### Data Dict v2: https://view.officeapps.live.com/op/view.aspx?src=https%3A%2F%2Fochin.org%2Fwp-content%2Fuploads%2F2024%2F04%2FADVANCE-and-OCHIN-DD-and-User-Guide.xlsx&wdOrigin=BROWSELINK

In [12]:
###### CONFIG #####
class eda_config:
    get_temporal_lab_results = False
    left_join_the_data  = True
    get_all_lab_features = True

In [13]:
### loading datasets ###
cohort_df = pl.scan_parquet("Determine_Cohort_with_labels_OutcomeDate.parquet")
medications_df = pl.scan_parquet("Determine_medications_cohort_data.parquet")

if eda_config.get_all_lab_features:
    lab_results_df = pl.scan_parquet('Determine_lab_results_cohort_data_all.parquet')
    
elif eda_config.get_temporal_lab_results:
    lab_results_df = pl.scan_parquet("Determine_lab_results_cohort_data_mean.parquet")

else:
    lab_results_df = pl.scan_parquet("Determine_lab_results_cohort_data_latest.parquet")

diagnoses_df = pl.scan_parquet("Determine_diagnoses_cohort_data.parquet")

demographics_df = pl.scan_parquet('Determine_demographics_cohort_data.parquet')
bmi_bP_df = pl.scan_parquet("Determine_dbmi_bp_cohort_data.parquet")
community_vital_df = pl.scan_parquet('Determine_community_vital_signs_cohort.parquet')
tob_df = pl.scan_parquet('Determine_tob_cohort_data.parquet')
height_df = pl.scan_parquet('Determine_height_cohort_data.parquet')
sdoh_screening_df = pl.scan_parquet('Determine_Cohort_sdoh_screening_questions.parquet')


pids_no_encs_after_vi_df = pl.scan_parquet('Determine_cohort_after_visit_index_details.parquet')

In [15]:
cohort_df.filter(pl.col('PATIENT_NUM').is_in([2085983,2255770])).collect()

PATIENT_NUM,Index_Start_date,FirstOutcomeDate,Age_at_encounter_date,Outcome,DEATH_DATE,Zip_CD
i64,datetime[μs],datetime[μs],i64,i64,datetime[μs],str
2255770,2016-10-07 00:00:00,2020-02-17 00:00:00,56,1,null,"""02121"""


In [16]:
### Count of unique patient records in each dataset
print("The number of unique patient records in cohort", len(np.unique(cohort_df.select(['PATIENT_NUM']).collect()['PATIENT_NUM'].to_list())))
print("The number of unique patient records in medications", len(np.unique(medications_df.select(['PATIENT_NUM']).collect()['PATIENT_NUM'].to_list())))
print("The number of unique patient records in lab results", len(np.unique(lab_results_df.select(['PATIENT_NUM']).collect()['PATIENT_NUM'].to_list())))
print("The number of unique patient records in diagnoses", len(np.unique(diagnoses_df.select(['PATIENT_NUM']).collect()['PATIENT_NUM'].to_list())))
#print("The number of unique patient records in sdoh", len(np.unique(cohort_df.select(['PATIENT_NUM']).collect()['PATIENT_NUM'].to_list())))

The number of unique patient records in cohort 1064468
The number of unique patient records in medications 531679
The number of unique patient records in lab results 428337
The number of unique patient records in diagnoses 751830


In [17]:
np.unique(cohort_df.select('Outcome').collect()['Outcome'].to_list(), return_counts = True)

(array([0, 1]), array([1004264,   60204]))

In [18]:
#### Number of patients that are present in atleast one of the medications, lab_results and diagnoses before pre-processin
cohort_pids = np.unique(cohort_df.select(['PATIENT_NUM']).collect()['PATIENT_NUM'].to_list())
med_pids = np.unique(medications_df.select(['PATIENT_NUM']).collect()['PATIENT_NUM'].to_list())
dx_pids = np.unique(diagnoses_df.select(['PATIENT_NUM']).collect()['PATIENT_NUM'].to_list())
lab_pid = np.unique(lab_results_df.select(['PATIENT_NUM']).collect()['PATIENT_NUM'].to_list())

# Convert lists to sets and use the union operation
result = set(med_pids) | set(dx_pids) | set(lab_pid) 

# result = result & set(cohort_pids)

# Convert the set back to a list if needed
result_list = list(result)

# Display the result
print("Number of patient ids that have data in atleast one of the medications, lab_results and diagnoses: ",len(result_list))


Number of patient ids that have data in atleast one of the medications, lab_results and diagnoses:  826658


In [19]:
## saving patient ids who have data across atleast one of the feature dataset
with open('determine_patient_ids_required.pkl','wb') as f:
    pickle.dump(result_list,f)
    

In [122]:
## Loading the patient ids who have data across atleast one of the feature dataset
with open('determine_patient_ids_required.pkl', 'rb') as file: 
    PID_REQ = pickle.load(file) 

In [21]:
np.unique(cohort_df.filter(pl.col('PATIENT_NUM').is_in(PID_REQ)).select('Outcome').collect()['Outcome'].to_list(), return_counts = True)

(array([0, 1]), array([786231,  40427]))

### EDA for medications
#### checks used:
##### 1. In function for retrieving active ingrident, response errro ( if  not response 200).
##### 2. Store list of RxNorms which dont have active ingridents associated with them.

In [ ]:
medications_df.collect().head()

In [ ]:
print("Number of data points: ",medications_df.collect().shape)

In [ ]:
## Number of unique medications ##
print("Number of unique medications: ",len(np.unique(medications_df.select(['RxNorm']).collect()['RxNorm'].to_list())))

In [ ]:
## Example usage for active ingrident##
rxcui = "16681"  # replace with actual RxNorm ID
active_ingredients = get_active_ingredient(rxcui)
print("Active Ingredients:", active_ingredients)

In [ ]:
rxcui = "7052"  # Example RxCUI
drug_classes = get_drug_class(rxcui)
print(f"Drug classes for RxCUI {rxcui}: {drug_classes}")

In [ ]:
## Loading mapping of rxnorm to active ingridents list
with open('rxnorm_active_ingridents_mapping.pkl', 'rb') as file: 
    dict_rxnorm_active_ing = pickle.load(file) 

In [ ]:
### Obtaining active ingridients for each unique rxnorm in dataframe ###
DONT_RUN = True
# Run only once to get active ingridents from API #

if DONT_RUN != True:
    ## Unique_rxnorms ##
    uniq_rxnorm = np.unique(medications_df.select(['RxNorm']).collect()['RxNorm'].to_list()).tolist()
    rxnorms_list = []
    uniq_active_ing = []


    for rxnorm in uniq_rxnorm:
        rx_code = rxnorm.split(':')[-1].strip()
        rxnorms_list.append(rx_code)
        if rx_code in dict_rxnorm_active_ing.items():
            uniq_active_ing.append(dict_rxnorm_active_ing[rx_code])
        else:
            active_ingredients = get_active_ingredient(rx_code)
            uniq_active_ing.append(active_ingredients)

    ## saving mapping of rxnorm to active ingridents list
    with open('rxnorm_active_ingridents_mapping.pkl','wb') as f:
        pickle.dump(dict(zip(rxnorms_list,uniq_active_ing)),f)

In [ ]:
get_active_ingredient(1359984)

In [ ]:
### mapping rxnorm to active ingredient in medications datafrane ###
no_act_ing_codes = []
rxnorms_dataframe_list = medications_df.select(['RxNorm']).collect()['RxNorm'].to_list()
active_ingredients_list_dataframe = []
for rxnorm in rxnorms_dataframe_list:
    rx_code = rxnorm.split(':')[-1].strip()
    try:
        if dict_rxnorm_active_ing[rx_code] == []:
            print(rx_code)
            no_act_ing_codes.append(rx_code)
            active_ingredients_list_dataframe.append(['rx_cui:'+str(rx_code)]) ### handling codes that dont have active ingredient associated with them
            continue
        active_ingredients_list_dataframe.append(dict_rxnorm_active_ing[rx_code])
    except:
        active_ingredients_list_dataframe.append(get_active_ingredient(rx_code))
        dict_rxnorm_active_ing[rx_code] = get_active_ingredient(rx_code)

In [ ]:
len(active_ingredients_list_dataframe)

In [ ]:
active_ingredients_list_dataframe[1]

In [ ]:
### Check:  if retrieved active ingrident list has [] elements
count = 0
for i in active_ingredients_list_dataframe:
    if i[0].startswith('rxcui'):
        count+=1
print(count)

In [ ]:
final_act_ing_list = ["_".join(sublist) if len(sublist) > 1 else sublist[0] for sublist in active_ingredients_list_dataframe]

In [ ]:
z = []
for i in dict_rxnorm_active_ing.values():
    if len(z) !=1:
        z.append('_'.join(i))
    else:
        z.append(i[0])

In [ ]:
print("Number of unique medications after active ingridents: ", len(np.unique(z)))

In [ ]:
def get_rx_code(element):
    return element.split(':')[-1].strip()

In [ ]:
## Adding active ingident data to dataframe
medications_df = medications_df.with_columns(pl.Series(name="Active_ingrident", values=final_act_ing_list),
                                            pl.col('RxNorm').map_elements(get_rx_code,return_dtype = pl.Utf8).alias('rxcui'))

In [ ]:
## filtering patients id based on PID_REQ
medications_df = medications_df.filter(pl.col('PATIENT_NUM').is_in(PID_REQ))
medications_df.collect().head()

In [ ]:
## ignoring medications shared by domain expert
ignore_rxnorm_codes = [
    "16681", "173", "1534763", "1368001", "1368384", "1368402", 
    "2627044", "1373458", "1545149", "2404", "1488564", "1486436", 
    "1727500", "1551291", "1545653", "1598392", "2281864", "1664314", 
    "1992672", "1992684", "1992825", "60548", "4816", "25789", 
    "647235", "606253", "4821", "352381", "25793", "4815", 
    "285129", "1007184", "51428", "1670007", "1727493", "139825", 
    "274783", "1858994", "400008", "631657", "1605101", "1008501", 
    "86009", "816726", "253182", "1100699", "1243019", "475968", 
    "1440051", "6809", "607999", "802646", "614348", "1043562", 
    "729717", "30009", "274332", "33738", "139953", "73044", 
    "84108", "857974", "1991302", "593411", "2638675", "2621880", 
    "2601723", "10633", "10635", "72610", "596554"
]

#medications_df = medications_df.filter(~pl.col('rxcui').is_in(ignore_rxnorm_codes))
act_list_to_drop = [get_active_ingredient(rxcui) for rxcui in ignore_rxnorm_codes]
medications_to_drop = ["_".join(sublist) if len(sublist) > 1 else sublist[0] for sublist in act_list_to_drop]

In [ ]:
print("Number of unique active ingridents to drop: ",len(medications_to_drop))

In [ ]:
## Obatin the patient_num with the active ingridents to drop later.

med_ignore_patient_num = np.unique(medications_df.filter(pl.col('Active_ingrident').is_in(medications_to_drop)).collect()['PATIENT_NUM'].to_list())
print("Number of patients to ignore: ",len(med_ignore_patient_num))

with open('./Determine_models_running_folder/pat_num_ignore_meds.pkl','wb') as f:
    pickle.dump(med_ignore_patient_num,f)

In [ ]:
medications_df = medications_df.drop("rxcui")

In [ ]:
medications_df.collect().write_parquet("./Determine_cohort_files/Determine_cohort_medications_active_ing.parquet")

In [ ]:
medications_df.collect().head(10)

### EDA for lab results

In [ ]:
######### lab results config ###########
class lab_config:
    use_only_domain_expert_features =  True
    use_all_features = True
    units_standardization =True
    
    get_median = True

In [ ]:
lab_results_df.collect().head()

## Check lab results on bp

In [ ]:
lab_results_df.filter(pl.col('LabLOINC').is_in(['LOINC:8480-6','LOINC:8462-4'])).collect()

## end bp check

In [ ]:
print("Number of unique lab loincs: ",len(np.unique(lab_results_df.select('LabLOINC').collect()['LabLOINC'].to_list())))

In [ ]:
lab_results_df = lab_results_df.filter(~pl.col('Result_Number').is_null())
lab_results_df = lab_results_df.with_columns(pl.col('Result_Number').cast(pl.Float32).alias('Result_Number'))

In [ ]:
lab_results_df.collect().shape

In [ ]:
### appending 'LOINC:' prefix for hem loincs
de_lr_features = ['LOINC:'+code for code in hem_loics]
de_lr_features[:3]

In [ ]:
unique_dataset_loincs = np.unique(lab_results_df.select('LabLOINC').collect()['LabLOINC'].to_list()).tolist()
    

In [ ]:
print(len(unique_dataset_loincs))

In [ ]:
'LOINC:2345-7' in unique_dataset_loincs

In [ ]:
unique_dataset_loincs[:5]

In [ ]:
lab_results_df = lab_results_df.collect()

#### Units check for each lab result

In [ ]:
lab_results_units_grouped_df = lab_results_df.group_by(['LabLOINC','Result_Unit']).agg([pl.col('Result_Number'),
                                                                                       pl.col('Result_Unit').count().alias('Unit_Count')
                                                                                       ]).sort(['LabLOINC','Result_Unit']).collect()

In [ ]:
print(len(lab_results_units_grouped_df))

In [ ]:
lab_results_units_grouped_df.filter(pl.col('LabLOINC') == 'LOINC:2345-7')

In [ ]:
Loincs_with_more_than_1_unit_count = 0
loinc_unit_pairs = []

for loinc in unique_dataset_loincs:
    focus_group_loinc = lab_results_units_grouped_df.filter(pl.col('LabLOINC') == loinc).sort('Unit_Count', descending=True)
    
    if len(focus_group_loinc) == 1:
        continue
        
    total_count_patients_associated_with = focus_group_loinc['Unit_Count'].sum()
    if total_count_patients_associated_with < 50:
        continue
    loinc_unit_with_max_count = focus_group_loinc['Result_Unit'][0]
    loinc_values_with_max_count = focus_group_loinc['Result_Number'][0].to_list()
    
    other_units_fg = focus_group_loinc.filter(pl.col('Result_Unit')!=loinc_unit_with_max_count)['Result_Unit'].to_list()
    max_val_in_pri_unit = max(loinc_values_with_max_count)
    min_val_in_pri_unit = min(loinc_values_with_max_count)
    mean_val_in_pri_unit = np.mean(loinc_values_with_max_count)
    
    print_flag =  False
    for unit in other_units_fg:
        
        other_unit_values = focus_group_loinc.filter(pl.col('Result_Unit') == unit)['Result_Number'][0].to_list()
#         print(other_unit_values)
        max_val_in_ou = max(other_unit_values)
        min_val_in_ou = min(other_unit_values)
        mean_val_in_ou = np.mean(other_unit_values)
        
        if min_val_in_pri_unit <= mean_val_in_ou <= max_val_in_pri_unit:
            continue
        else:
            loinc_unit_pairs.append((loinc,unit))
            print("Unit to focus: ",unit)            
            Loincs_with_more_than_1_unit_count += 1
            if mean_val_in_pri_unit/mean_val_in_ou > 10 or mean_val_in_pri_unit/mean_val_in_ou < 0.1:
                print(" The value is orders above 10th order")
                print_flag = True
            else:
                print(" The value is orders below 10th order")
                print_flag = True
                
    if print_flag:
        print(focus_group_loinc)
                
    print("*"*20)
            
        
        
    
    

In [ ]:
loinc_unit_pairs[:2]


In [ ]:
combo_df.head()

In [ ]:
if lab_config.units_standardization:
    ### removing loinc and unit pairs that failed above code
    combo_df = pl.DataFrame(loinc_unit_pairs, schema=['LabLOINC', 'Result_Unit'])
    lab_results_df = lab_results_df.filter(~pl.struct((pl.col('LabLOINC'),pl.col('Result_Unit'))).
                                           is_in(combo_df.select(pl.struct(['LabLOINC', 'Result_Unit']))))

In [ ]:
lab_results_df.shape

In [ ]:
if lab_config.get_median:
    lab_results_df = lab_results_df.group_by(['PATIENT_NUM','LabLOINC']).agg(pl.col('Result_Number').median().alias('Result_Number'))

In [ ]:
lab_results_df.shape

##### Creating two different feature sets (  domain expert vs all features)

In [ ]:
if lab_config.use_only_domain_expert_features:
    unique_dataset_loincs = np.unique(lab_results_df.select('LabLOINC')['LabLOINC'].to_list()).tolist()
    common_loincs = list(set(de_lr_features) & set(unique_dataset_loincs))
    print('Number of domain expert features present in dataset',len(common_loincs))
    lab_results_de_loincs_df = lab_results_df.filter(pl.col("LabLOINC").is_in(common_loincs))
    
    if lab_config.get_median:
        lab_results_de_loincs_df.write_parquet("./Determine_cohort_files/Determine_cohort_labresults_domain_expert_median.parquet")
        
if lab_config.use_all_features:
    unique_dataset_loincs = np.unique(lab_results_df.select('LabLOINC')['LabLOINC'].to_list()).tolist()
    print("Number of features in the dataset: ",len(unique_dataset_loincs))
    lab_results_df.write_parquet("./Determine_cohort_files/Determine_cohort_labresults_all_features_median.parquet")

#### Preparing different configurations of lab results (latest, mean and temporal[latest, oldest, mean])

In [ ]:
del lab_results_df,lab_results_de_loincs_df

### EDA for diagnoses

In [37]:
class diag_config:
    use_phecodes =  True
    use_n_chars = True
    n_icd_chars = 3
    

In [17]:
# Phecodes
icd10_phe = pl.read_csv('./icd_code_to_phecode/Phecode_map_v1_2_icd10cm_beta.csv')
icd9_phe_v1 =  pl.read_csv('./icd_code_to_phecode/phecode_map_v1_2_icd9.csv',infer_schema_length=100000)
icd9_phe_v2 = pyreadr.read_r("./icd_code_to_phecode/phemap (1).rda")['phemap'].drop_duplicates(subset=['icd9'])

def read_icd_mappings(file_path):
    icd_mapping = {}

    # Open the file and read it line by line
    with open(file_path, 'r') as file:
        for line in file:
            # Strip any white space and split by the delimiter '|'
            parts = line.strip().split('|')
            if len(parts) >= 2:
                icd9_code = parts[0]
                icd10_code = parts[1]

                # Store the mapping in the dictionary
                icd_mapping[icd9_code] = icd10_code

    return icd_mapping

file_path = './icd_code_to_phecode/icd9to10dictionary.txt'
icd9_to_10_mapping_dict = read_icd_mappings(file_path)

In [31]:
icd9_to_10_mapping_dict['368.8']

'H53.8'

In [18]:
### dicts for mapping ###
icd10_phe_dict = dict(zip(icd10_phe['ICD10CM'].to_list(), icd10_phe['PHECODE'].to_list()))
icd9_phe_dict_v1 = dict(zip(icd9_phe_v1['ICD9'].to_list(), icd9_phe_v1['PheCode'].to_list()))
icd9_phe_dict_v2 = dict(zip(icd9_phe_v2['icd9'].to_list(), icd9_phe_v2['phecode'].to_list()))

In [19]:
diagnoses_df.head().collect()

PATIENT_NUM,ENCOUNTER_NUM,START_DATE,CONCEPT_CD
i64,i64,datetime[μs],str
556,27844565,2016-02-26 00:00:00,"""ICD10CM:Z23"""
556,27844565,2016-02-26 00:00:00,"""ICD10CM:Z13.89"""
556,27844565,2016-02-26 00:00:00,"""ICD10CM:Z13.220"""
556,28287946,2016-03-23 00:00:00,"""ICD10CM:Z12.4"""
556,28287946,2016-03-23 00:00:00,"""ICD10CM:Z12.39"""


In [57]:
from icdmappings import Mapper

mapper = Mapper()
icd9_code = "8290"  # Example ICD-9 code
icd10_codes = mapper.map(icd9_code, source='icd9', target='icd10')
if icd10_codes:
    print(f"ICD-10 codes for {icd9_code}: {icd10_codes}")
else:
    print(f"No ICD-10 codes found for {icd9_code}")

ICD-10 codes for 8290: T148


In [100]:
def convert_icd9_to_icd10(icd9):
    mapper = Mapper()
    icd10_code = mapper.map(icd9_code, source='icd9', target='icd10')
    if icd10_code:
        return icd10_code[:3]+'.'+icd10_code[3:]
    else:
        return 'ICD9CM:'+icd9

def standardize_concept_cd(code):
    if code.startswith('ICD10CM'):
        return code.split(':')[-1]
    elif code.startswith('ICD9CM'):
#      return convert_icd9_to_icd10(code.split(':')[-1].replace('.',''))
       try:
            return icd9_to_10_mapping_dict[code.split(':')[-1]] 
       except:
            return convert_icd9_to_icd10(code.split(':')[-1].replace('.',''))
    else:
        return code
    
def get_first_n_char_icdcode(std_icd_code, n = diag_config.n_icd_chars):
    if std_icd_code.startswith('ICD9'):
        return std_icd_code
    else:
        if len(std_icd_code)>3:
            new_code = std_icd_code[:5]
            return new_code[:n]
        else:
            return std_icd_code
        
def get_phecode_from_concept_cd(cd):
    if cd.startswith('ICD10CM'):
        
        if cd.startswith("ICD10CM:Z"):
            return cd.split('.')[0]
        
        icd_code = cd.split(':')[-1]
        try:
            phecode = str(icd10_phe_dict[icd_code])
            if phecode  =='1010.0':
                return cd.split('.')[0]
            return 'phe_'+phecode
        except:
            
            return cd.split('.')[0]
        
    elif cd.startswith('ICD9CM'):
        icd_code = cd.split(':')[-1]
        try:
            phecode = str(icd9_phe_dict_v2[icd_code])
            if phecode =='1010.0':
                        return cd.split('.')[0]
            return 'phe_'+phecode
        except:
            try:
                phecode  = str(icd9_phe_dict_v1[icd_code])
                if phecode =='1010.0':
                        return cd.split('.')[0]
                return 'phe_'+phecode
            
            except:
                try: #convert icd9 code to icd10 and try phemap
                    phecode = icd10_phe_dict(icd9_to_10_mapping_dict(icd_code))
                    if phecode =='1010.0':
                        return cd.split('.')[0]
                    return 'phe_'+phecode
                except:
                    
                    return cd.split('.')[0]
    else:
        return cd
# def get_phe_code()

In [63]:
diagnoses_df.collect().head()

PATIENT_NUM,ENCOUNTER_NUM,START_DATE,CONCEPT_CD
i64,i64,datetime[μs],str
556,27844565,2016-02-26 00:00:00,"""ICD10CM:Z23"""
556,27844565,2016-02-26 00:00:00,"""ICD10CM:Z13.89"""
556,27844565,2016-02-26 00:00:00,"""ICD10CM:Z13.220"""
556,28287946,2016-03-23 00:00:00,"""ICD10CM:Z12.4"""
556,28287946,2016-03-23 00:00:00,"""ICD10CM:Z12.39"""


In [64]:
print("Number of unique ICD codes: ", len(np.unique(diagnoses_df.select('CONCEPT_CD').collect()['CONCEPT_CD'].to_list())))
print("Number of unique ICD 10 codes: ", len(np.unique(diagnoses_df.select('CONCEPT_CD').filter(pl.col('CONCEPT_CD').str.contains('ICD10')).collect()['CONCEPT_CD'].to_list())))
print("Number of unique ICD 9 codes: ", len(np.unique(diagnoses_df.select('CONCEPT_CD').filter(pl.col('CONCEPT_CD').str.contains('ICD9')).collect()['CONCEPT_CD'].to_list())))

Number of unique ICD codes:  28180
Number of unique ICD 10 codes:  23638
Number of unique ICD 9 codes:  4542


In [65]:
icd9_codes = np.unique(diagnoses_df.select('CONCEPT_CD').filter(pl.col('CONCEPT_CD').str.contains('ICD9')).collect()['CONCEPT_CD'].to_list())

In [66]:
len(icd9_codes)

4542

### Initial 4 chars for diagnoses

In [67]:
if diag_config.use_n_chars:
    diagnoses_df_n_chars = diagnoses_df.with_columns(pl.col('CONCEPT_CD').map_elements(standardize_concept_cd,return_dtype = pl.Utf8).alias('standardized_icd10_codes')).collect()
    diagnoses_df_n_chars = diagnoses_df_n_chars.with_columns(pl.col('standardized_icd10_codes').map_elements(get_first_n_char_icdcode,return_dtype = pl.Utf8).alias('get_first_'+str(diag_config.n_icd_chars)+'_char_icdcode'))

In [68]:
diagnoses_df_n_chars.head()

PATIENT_NUM,ENCOUNTER_NUM,START_DATE,CONCEPT_CD,standardized_icd10_codes,get_first_3_char_icdcode
i64,i64,datetime[μs],str,str,str
556,27844565,2016-02-26 00:00:00,"""ICD10CM:Z23""","""Z23""","""Z23"""
556,27844565,2016-02-26 00:00:00,"""ICD10CM:Z13.89""","""Z13.89""","""Z13"""
556,27844565,2016-02-26 00:00:00,"""ICD10CM:Z13.220""","""Z13.220""","""Z13"""
556,28287946,2016-03-23 00:00:00,"""ICD10CM:Z12.4""","""Z12.4""","""Z12"""
556,28287946,2016-03-23 00:00:00,"""ICD10CM:Z12.39""","""Z12.39""","""Z12"""


In [77]:
diagnoses_df_n_chars.filter(pl.col('get_first_3_char_icdcode').str.starts_with('E13'))

PATIENT_NUM,ENCOUNTER_NUM,START_DATE,CONCEPT_CD,standardized_icd10_codes,get_first_3_char_icdcode
i64,i64,datetime[μs],str,str,str
2051057,2230852,2015-08-07 00:00:00,"""ICD9CM:357.2""","""E13.42""","""E13"""
2951472,40858180,2015-06-05 00:00:00,"""ICD9CM:357.2""","""E13.42""","""E13"""
2951472,40441142,2015-07-08 00:00:00,"""ICD9CM:357.2""","""E13.42""","""E13"""


In [78]:
diagnoses_df_n_chars.select(['get_first_3_char_icdcode']).collect()['get_first_3_char_icdcode']

AttributeError: 'DataFrame' object has no attribute 'collect'

In [46]:
print(type(diagnoses_df_n_chars))

<class 'polars.lazyframe.frame.LazyFrame'>


In [79]:
diagnoses_df_n_chars.null_count()

PATIENT_NUM,ENCOUNTER_NUM,START_DATE,CONCEPT_CD,standardized_icd10_codes,get_first_3_char_icdcode
u32,u32,u32,u32,u32,u32
0,0,0,0,0,0


In [80]:
diagnoses_df_n_chars.filter(pl.col('standardized_icd10_codes') == '831.04')

PATIENT_NUM,ENCOUNTER_NUM,START_DATE,CONCEPT_CD,standardized_icd10_codes,get_first_3_char_icdcode
i64,i64,datetime[μs],str,str,str


In [83]:
if diag_config.use_n_chars:
    diagnoses_df_n_chars = diagnoses_df_n_chars.drop_nulls()
    print("Number of unique ICD codes after initial 4 chars: ",len(np.unique(diagnoses_df_n_chars.select('get_first_'+str(diag_config.n_icd_chars)+'_char_icdcode')['get_first_'+str(diag_config.n_icd_chars)+'_char_icdcode'].to_list())))
    diagnoses_df_n_chars.write_parquet('./Determine_cohort_files/Determine_cohort_diagnoses_initail_'+str(diag_config.n_icd_chars)+'_chars.parquet')

Number of unique ICD codes after initial 4 chars:  1719


### Phecodes_v1 (legacy)

In [ ]:
icd10_phecode_map.head()

In [ ]:
len(np.unique(icd10_phecode_map['PHECODE'].to_list()))

In [ ]:
icd10cm_to_phecode_dict = dict(zip(icd10_phecode_map['ICD10CM'].to_list(), icd10_phecode_map['PHECODE'].to_list()))

In [ ]:
def icd10_to_phecode_map_func(icd10code):
    if icd10code in list(icd10cm_to_phecode_dict.keys()):
        return str(icd10cm_to_phecode_dict[icd10code])
    else:
        return ""str(icd10code)

In [ ]:
diagnoses_df = diagnoses_df.with_columns(pl.col('CONCEPT_CD').map_elements(standardize_concept_cd,return_dtype = pl.Utf8).alias('standardized_icd10_codes'))

In [ ]:
diagnoses_df.collect().head()

In [ ]:
diagnoses_df = diagnoses_df.with_columns(
    pl.when(pl.col('standardized_icd10_codes').is_in(list(icd10cm_to_phecode_dict.keys())))
    .then(pl.col('standardized_icd10_codes').replace(icd10cm_to_phecode_dict))
    .otherwise(pl.col('CONCEPT_CD'))
    .cast(pl.Utf8)
    .alias('phecode_map')
)

In [ ]:
diagnoses_df.collect().head()

In [ ]:
diagnoses_df.null_count().collect()

In [ ]:
diagnoses_df = diagnoses_df.drop_nulls()

In [ ]:
print("Number if unique codes after phecode map: ",len(np.unique(diagnoses_df.filter(~pl.col('standardized_icd10_codes').is_null())
                                                                 .select('phecode_map').collect()['phecode_map'].to_list())))

In [ ]:
print("Number if unique codes which cannot be converted after phecode map: ",len(np.unique(diagnoses_df.filter(pl.col('phecode_map').str.starts_with('ICD'))
                                                                .select('phecode_map').collect()['phecode_map'].to_list())))

In [ ]:
diagnoses_df.collect().write_parquet("Determine_cohort_diagnoses_phecode_map.parquet")

### Phecode v2 (Standardizing from ICD9 and ICD10)

In [101]:
diagnoses_df = diagnoses_df.with_columns(pl.col('CONCEPT_CD').map_elements(get_phecode_from_concept_cd,return_dtype = pl.Utf8).alias('phecode_map'))

In [102]:
diagnoses_df.collect().head()

PATIENT_NUM,ENCOUNTER_NUM,START_DATE,CONCEPT_CD,phecode_map
i64,i64,datetime[μs],str,str
556,27844565,2016-02-26 00:00:00,"""ICD10CM:Z23""","""ICD10CM:Z23"""
556,27844565,2016-02-26 00:00:00,"""ICD10CM:Z13.89""","""ICD10CM:Z13"""
556,27844565,2016-02-26 00:00:00,"""ICD10CM:Z13.220""","""ICD10CM:Z13"""
556,28287946,2016-03-23 00:00:00,"""ICD10CM:Z12.4""","""ICD10CM:Z12"""
556,28287946,2016-03-23 00:00:00,"""ICD10CM:Z12.39""","""ICD10CM:Z12"""


In [103]:
diagnoses_df.null_count().collect()

PATIENT_NUM,ENCOUNTER_NUM,START_DATE,CONCEPT_CD,phecode_map
u32,u32,u32,u32,u32
0,0,0,0,0


In [104]:
diag_null_phe_df = diagnoses_df.filter(pl.col('phecode_map').is_null())

In [105]:
len(np.unique(diag_null_phe_df.filter(pl.col('CONCEPT_CD').str.starts_with('ICD10')).collect()['CONCEPT_CD'].to_list()))

0

In [106]:
diagnoses_df.filter(#(pl.col('phecode_map').str.starts_with('1010')) & 
                   (pl.col('CONCEPT_CD').str.starts_with('ICD10CM:Z68.3'))).collect()

PATIENT_NUM,ENCOUNTER_NUM,START_DATE,CONCEPT_CD,phecode_map
i64,i64,datetime[μs],str,str
823,2498907,2017-03-13 00:00:00,"""ICD10CM:Z68.38""","""ICD10CM:Z68"""
1016,2456176,2016-07-15 00:00:00,"""ICD10CM:Z68.31""","""ICD10CM:Z68"""
1016,1964356,2016-08-25 00:00:00,"""ICD10CM:Z68.31""","""ICD10CM:Z68"""
1213,60490975,2016-07-28 00:00:00,"""ICD10CM:Z68.30""","""ICD10CM:Z68"""
1390,60086251,2016-05-25 00:00:00,"""ICD10CM:Z68.31""","""ICD10CM:Z68"""
…,…,…,…,…
12319299,0,2016-05-10 00:00:00,"""ICD10CM:Z68.33""","""ICD10CM:Z68"""
12365454,0,2016-05-31 00:00:00,"""ICD10CM:Z68.39""","""ICD10CM:Z68"""
12378183,0,2016-03-04 00:00:00,"""ICD10CM:Z68.38""","""ICD10CM:Z68"""


In [107]:
icd_codes_not_phe = np.unique(diagnoses_df.filter(#(pl.col('phecode_map').str.starts_with('1010')) & 
                   (pl.col('phecode_map').str.starts_with('ICD'))).collect()['phecode_map'].to_list())

In [108]:
len(icd_codes_not_phe)

631

In [109]:
icd_codes_not_phe[:500]

array(['ICD10CM:A04', 'ICD10CM:A41', 'ICD10CM:A52', 'ICD10CM:A54',
       'ICD10CM:A56', 'ICD10CM:A63', 'ICD10CM:A74', 'ICD10CM:A88',
       'ICD10CM:A92', 'ICD10CM:B08', 'ICD10CM:B15', 'ICD10CM:B17',
       'ICD10CM:B18', 'ICD10CM:B95', 'ICD10CM:B96', 'ICD10CM:B97',
       'ICD10CM:C44', 'ICD10CM:D32', 'ICD10CM:D35', 'ICD10CM:D47',
       'ICD10CM:D48', 'ICD10CM:D53', 'ICD10CM:D57', 'ICD10CM:D63',
       'ICD10CM:D68', 'ICD10CM:D75', 'ICD10CM:E01', 'ICD10CM:E03',
       'ICD10CM:E04', 'ICD10CM:E22', 'ICD10CM:E23', 'ICD10CM:E27',
       'ICD10CM:E61', 'ICD10CM:E66', 'ICD10CM:E71', 'ICD10CM:E72',
       'ICD10CM:E74', 'ICD10CM:E78', 'ICD10CM:E80', 'ICD10CM:E87',
       'ICD10CM:E88', 'ICD10CM:E89', 'ICD10CM:F06', 'ICD10CM:F10',
       'ICD10CM:F11', 'ICD10CM:F12', 'ICD10CM:F13', 'ICD10CM:F14',
       'ICD10CM:F15', 'ICD10CM:F16', 'ICD10CM:F17', 'ICD10CM:F19',
       'ICD10CM:F34', 'ICD10CM:F41', 'ICD10CM:F43', 'ICD10CM:F48',
       'ICD10CM:F51', 'ICD10CM:F52', 'ICD10CM:F53', 'ICD10CM:F

In [110]:
def get_initial_char_for_icd_codes(code):
    if code.startswith("ICD10CM"):
        return code.split('.')[0]
    else:
        return code

In [111]:
diagnoses_df = diagnoses_df.with_columns(pl.col("phecode_map").map_elements(get_initial_char_for_icd_codes, return_dtype = pl.Utf8).alias("phecode_map"))

In [112]:
icd10_codes = diagnoses_df.filter(pl.col('phecode_map').str.starts_with('ICD10CM:')).collect()['phecode_map'].to_list()
first_characters = [code.split(':')[1].split('.')[0] for code in icd10_codes]
print("Number of phecodes that start with ICD10: ,", len(np.unique(icd10_codes)))

Number of phecodes that start with ICD10: , 475


In [113]:
icd9_codes = diagnoses_df.filter(pl.col('phecode_map').str.starts_with('ICD9CM')).collect()['phecode_map'].to_list()
#first_characters = [code.split(':')[1].split('.')[0] for code in icd10_codes]
print("Number of phecodes that start with ICD9: ,", len(np.unique(icd9_codes)))

Number of phecodes that start with ICD9: , 154


In [114]:
print("Number if unique codes after phecode map: ",len(np.unique(diagnoses_df
                                                                 .select('phecode_map').collect()['phecode_map'].to_list())))

Number if unique codes after phecode map:  2810


#### Creating two diagnoses datasets one with Z codes other without these codes

In [115]:
diagnoses_df.collect().write_parquet("./Determine_cohort_files/Determine_cohort_diagnoses_phecode_map_v2_with_z_codes.parquet")

In [116]:
diagnoses_df_without_icd10z = diagnoses_df.filter(~pl.col('phecode_map').str.starts_with('ICD10CM:Z'))
diagnoses_df_without_icd10z.collect().write_parquet("./Determine_cohort_files/Determine_cohort_diagnoses_phecode_map_v2_without_z_codes.parquet")

### Archive code for diagnoses

In [ ]:
diagnoses_df

In [ ]:
icd10_phecode_map.shape

In [ ]:
#### ICD 10 codes from phecodes ###
phe_icd10_codes = icd10_phecode_map['ICD10CM'].to_list()

In [ ]:
### ICD 10 codes from dataframe ###
icd10_from_df = np.unique(diagnoses_df.filter(~pl.col('standardized_icd10_codes').is_null())
                          .select('standardized_icd10_codes')
                          .collect()['standardized_icd10_codes'].to_list())

In [ ]:
icd10_from_df

In [ ]:
common_elements = np.intersect1d(icd10_from_df, phe_icd10_codes)

In [ ]:
len(common_elements)

In [ ]:
len(icd10_from_df)

In [ ]:
# Find elements in `sample_array` that are not in `list_set`
elements_in_array_not_in_list = np.setdiff1d(icd10_from_df, phe_icd10_codes)

In [ ]:
len(elements_in_array_not_in_list)

In [ ]:
elements_in_array_not_in_list[:10]

### EDA for target variable

In [117]:
cohort_df.collect().head()

PATIENT_NUM,Index_Start_date,FirstOutcomeDate,Age_at_encounter_date,Outcome,DEATH_DATE,Zip_CD
i64,datetime[μs],datetime[μs],i64,i64,datetime[μs],str
8202921,2017-03-15 00:00:00,2019-05-29 00:00:00,60,1,null,"""83706"""
6046573,2017-03-27 00:00:00,null,45,0,null,"""97136"""
2179286,2016-12-13 00:00:00,null,34,0,null,"""77530"""
6861988,2016-09-16 00:00:00,null,33,0,null,"""90035"""
7452088,2017-02-02 00:00:00,null,43,0,null,"""75217"""


In [118]:
list_outcome_years = cohort_df.filter(~pl.col('FirstOutcomeDate').is_null()).with_columns(
        pl.col("FirstOutcomeDate").dt.year().alias("year_from_date")
    ).select('year_from_date').collect()['year_from_date'].to_list()

In [119]:
np.unique(list_outcome_years, return_counts = True)

(array([2016, 2017, 2018, 2019, 2020, 2021, 2022]),
 array([  372, 10165, 13982, 16817,  9417,  9592,  8007]))

In [123]:
cohort_df = cohort_df.filter(pl.col('PATIENT_NUM').is_in(PID_REQ))

In [124]:
cohort_df.collect().head()

PATIENT_NUM,Index_Start_date,FirstOutcomeDate,Age_at_encounter_date,Outcome,DEATH_DATE,Zip_CD
i64,datetime[μs],datetime[μs],i64,i64,datetime[μs],str
6046573,2017-03-27 00:00:00,null,45,0,null,"""97136"""
2179286,2016-12-13 00:00:00,null,34,0,null,"""77530"""
6861988,2016-09-16 00:00:00,null,33,0,null,"""90035"""
576517,2017-03-24 00:00:00,null,47,0,null,"""97031"""
807301,2017-03-08 00:00:00,null,32,0,null,"""98023"""


In [125]:
### Transforming the age feature to groups

In [126]:
cohort_df = cohort_df.with_columns(pl.col('Age_at_encounter_date').map_elements(lambda x : '18-34' if x <= 34 \
                                                        else '35-44' if 35<=x<=44 else '45-54'\
                                                        if 45<=x<=54 else '55-64' if 55<=x<=64 else \
                                                        '65-74' if  65<=x<=74 else '75_older').alias('Age_group'))

In [127]:
cohort_df.collect().head()

sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.


PATIENT_NUM,Index_Start_date,FirstOutcomeDate,Age_at_encounter_date,Outcome,DEATH_DATE,Zip_CD,Age_group
i64,datetime[μs],datetime[μs],i64,i64,datetime[μs],str,str
6046573,2017-03-27 00:00:00,null,45,0,null,"""97136""","""45-54"""
2179286,2016-12-13 00:00:00,null,34,0,null,"""77530""","""18-34"""
6861988,2016-09-16 00:00:00,null,33,0,null,"""90035""","""18-34"""
576517,2017-03-24 00:00:00,null,47,0,null,"""97031""","""45-54"""
807301,2017-03-08 00:00:00,null,32,0,null,"""98023""","""18-34"""


## EDA for demographics 

In [128]:
demographics_df.collect().head()

Patient_num,Sex_CD,Race_CD,Hispanic_CD,Gender_CD
i64,str,str,str,str
114343,"""DEM|SEX:M""","""DEM|RACE:05""","""DEM|HISPANIC:N""","""DEM|GENDER:M"""
114400,"""DEM|SEX:F""","""DEM|RACE:05""","""DEM|HISPANIC:N""","""DEM|GENDER:W"""
114403,"""DEM|SEX:M""","""DEM|RACE:03""","""DEM|HISPANIC:N""","""DEM|GENDER:UN"""
114406,"""DEM|SEX:F""","""DEM|RACE:07""","""DEM|HISPANIC:Y""","""DEM|GENDER:W"""
114407,"""DEM|SEX:F""","""DEM|RACE:03""","""DEM|HISPANIC:N""","""DEM|GENDER:UN"""


In [129]:
np.unique(demographics_df.select('Gender_CD').collect()['Gender_CD'].to_list())

array(['DEM|GENDER:GQ', 'DEM|GENDER:M', 'DEM|GENDER:OT', 'DEM|GENDER:TG',
       'DEM|GENDER:UN', 'DEM|GENDER:W'], dtype='<U13')

In [130]:
PID_REQ[:4]

[8388608, 6291456, 6291458, 8388613]

In [131]:
demographics_df = demographics_df.filter(pl.col('Patient_num').is_in(PID_REQ))

In [132]:
def clean_prefix_data(value):
    return value.split(':')[-1]

In [133]:
demographics_df = demographics_df.with_columns( pl.col('Sex_CD').map_elements(clean_prefix_data, return_dtype = pl.Utf8).alias('Sex_CD'),
                                               pl.col('Race_CD').map_elements(clean_prefix_data, return_dtype = pl.Utf8).alias('Race_CD'),
                                               pl.col('Hispanic_CD').map_elements(clean_prefix_data, return_dtype = pl.Utf8).alias('Hispanic_CD'),
                                               pl.col('Gender_CD').map_elements(clean_prefix_data, return_dtype = pl.Utf8).alias('Gender_CD')
                                  )

In [134]:
demographics_df.head().collect()

Patient_num,Sex_CD,Race_CD,Hispanic_CD,Gender_CD
i64,str,str,str,str
114343,"""M""","""05""","""N""","""M"""
114406,"""F""","""07""","""Y""","""W"""
114410,"""F""","""05""","""N""","""W"""
114413,"""M""","""05""","""Y""","""UN"""
114440,"""M""","""05""","""N""","""UN"""


In [135]:
race_codes = {
    "01": "American Indian or Alaska Native",
    "02": "Asian",
    "03": "Black or African American",
    "04": "Native Hawaiian or Other Pacific Islander",
    "05": "White",
    "06": "Multiple race",
    "07": "Refuse to answer",
    "NI": "No information",
    "OT": "Other",
    "UN": "Unknown/missing"
}

result = (
    demographics_df
    .group_by('Race_CD')
    .agg([
        pl.count('Race_CD').alias('count'),
        (pl.count('Race_CD') / len(demographics_df.collect()) * 100).alias('percentage')
    ])
)

result.collect()

Race_CD,count,percentage
str,u32,f64
"""NI""",2849,0.344641
"""UN""",56292,6.809588
"""04""",5281,0.638837
"""07""",19339,2.33942
"""01""",8164,0.987591
"""05""",518922,62.773481
"""03""",157652,19.071006
"""OT""",865,0.104638
"""06""",9006,1.089447


In [136]:
result.with_columns(pl.col('Race_CD').replace_strict(race_codes).alias('Race_CD')).collect()

Race_CD,count,percentage
str,u32,f64
"""Black or African American""",157652,19.071006
"""White""",518922,62.773481
"""No information""",2849,0.344641
"""American Indian or Alaska Nati…",8164,0.987591
"""Other""",865,0.104638
"""Native Hawaiian or Other Pacif…",5281,0.638837
"""Refuse to answer""",19339,2.33942
"""Multiple race""",9006,1.089447
"""Asian""",48288,5.841352


In [137]:
np.unique(demographics_df.select('Race_CD').collect()['Race_CD'].to_list())

array(['01', '02', '03', '04', '05', '06', '07', 'NI', 'OT', 'UN'],
      dtype='<U2')

In [138]:
# Values to be replaced
values_to_replace = ["06", "07"]
replacement_value = "UN"

# Replace multiple values with a single value
demographics_df = demographics_df.with_columns(
    pl.when(pl.col("Race_CD").is_in(values_to_replace))
    .then(pl.lit(replacement_value))
    .otherwise(pl.col("Race_CD"))
    .alias("Race_CD"))

In [139]:
demographics_df.collect().head()

Patient_num,Sex_CD,Race_CD,Hispanic_CD,Gender_CD
i64,str,str,str,str
114343,"""M""","""05""","""N""","""M"""
114406,"""F""","""UN""","""Y""","""W"""
114410,"""F""","""05""","""N""","""W"""
114413,"""M""","""05""","""Y""","""UN"""
114440,"""M""","""05""","""N""","""UN"""


In [140]:
demographics_df.collect().write_parquet('./Determine_cohort_files/Determine_modeling_demographics_data.parquet')

## EDA for BMI and BP

### BMI (Legacy)

In [ ]:
bmi_bP_df.collect().head()

In [ ]:
bmi_bP_df = bmi_bP_df.cast({'NVAL_NUM': pl.Float32})

In [ ]:
### filtering BMI data
bmi_df = bmi_bP_df.filter(pl.col('CONCEPT_CD').str.contains('VIT\|BMI'))

In [ ]:
bmi_df.collect().head()

In [ ]:
bmi_df.collect().null_count()

In [ ]:
print("Number of unique patients: ",len(np.unique(bmi_df.select('PATIENT_NUM').collect()['PATIENT_NUM'].to_list())))

In [ ]:
print("Number of records: ",bmi_df.collect().shape)

In [ ]:
print("Number of unique BMI CONCEPT_CD ranges : ", np.unique(bmi_df.select('CONCEPT_CD').collect()['CONCEPT_CD'].to_list()))

In [ ]:
print("Number of unique measuring units : ", np.unique(bmi_df.select('UNITS_CD').collect()['UNITS_CD'].to_list()))

In [ ]:
bmi_list = bmi_df.select('NVAL_NUM').collect()['NVAL_NUM'].to_list()
print("Min and max of BMI: ", min(bmi_list), max(bmi_list))

In [ ]:
bmi_less_80_df = bmi_df.filter(pl.col('NVAL_NUM') < 80)

In [ ]:
len(np.unique(bmi_less_80_df.select('PATIENT_NUM').collect()['PATIENT_NUM'].to_list()))

## Verifying BMI using height and weight

### Weight

In [157]:
ht_wt_df = bmi_bP_df.filter(pl.col('CONCEPT_CD').str.contains('WT'))

In [158]:
ht_wt_df = ht_wt_df.with_columns(pl.col("NVAL_NUM").cast(pl.Float32))

In [159]:
print("Number of unique pid", len(np.unique(ht_wt_df.select('PATIENT_NUM').collect()['PATIENT_NUM'].to_list())))

Number of unique pid 665713


In [160]:
print("Number of unique units", len(np.unique(ht_wt_df.select('UNITS_CD').collect()['UNITS_CD'].to_list())))

Number of unique units 1


In [161]:
ht_wt_df.collect().filter(pl.col('NVAL_NUM') > 500).sort('PATIENT_NUM','NVAL_NUM')

PATIENT_NUM,CONCEPT_CD,NVAL_NUM,UNITS_CD,START_DATE
i64,str,f32,str,datetime[μs]
43530,"""VIT|WT:>499""",564.0,"""Lb""",2017-02-10 00:00:00
43530,"""VIT|WT:>499""",571.799988,"""Lb""",2016-06-07 00:00:00
43530,"""VIT|WT:>499""",581.0,"""Lb""",2016-05-12 00:00:00
43530,"""VIT|WT:>499""",581.799988,"""Lb""",2016-08-02 00:00:00
63622,"""VIT|WT:>499""",508.0,"""Lb""",2016-12-13 00:00:00
…,…,…,…,…
11275054,"""VIT|WT:>499""",1458.800049,"""Lb""",2016-09-24 00:00:00
11341172,"""VIT|WT:>499""",786.599976,"""Lb""",2016-03-07 00:00:00
11377332,"""VIT|WT:>499""",523.0,"""Lb""",2017-01-31 00:00:00


In [164]:
filtered = (
    ht_wt_df
    .group_by("PATIENT_NUM")
    .agg([
        pl.count("NVAL_NUM").alias("count"),
        pl.mean("NVAL_NUM").alias("mean_value"),
        pl.min("NVAL_NUM").alias("min_value"),
        pl.max("NVAL_NUM").alias("max_value")
    ])
    .filter((pl.col("count") > 2) & ((pl.col("max_value") - pl.col("mean_value")) > 50))
)

# Join back to original LazyFrame if needed
# result = ht_wt_df.join(filtered, on="PATIENT_NUM")

# # Collect the results
# final_result = result.collect()

print(filtered.collect())

shape: (618, 5)
┌─────────────┬───────┬────────────┬────────────┬────────────┐
│ PATIENT_NUM ┆ count ┆ mean_value ┆ min_value  ┆ max_value  │
│ ---         ┆ ---   ┆ ---        ┆ ---        ┆ ---        │
│ i64         ┆ u32   ┆ f32        ┆ f32        ┆ f32        │
╞═════════════╪═══════╪════════════╪════════════╪════════════╡
│ 5460790     ┆ 8     ┆ 168.625    ┆ 132.5      ┆ 402.0      │
│ 6989323     ┆ 4     ┆ 439.734985 ┆ 371.200012 ┆ 637.0      │
│ 4537203     ┆ 6     ┆ 234.0      ┆ 219.0      ┆ 292.0      │
│ 7624920     ┆ 9     ┆ 382.022217 ┆ 186.600006 ┆ 1906.0     │
│ 17968       ┆ 6     ┆ 256.75     ┆ 240.0      ┆ 340.0      │
│ …           ┆ …     ┆ …          ┆ …          ┆ …          │
│ 6477170     ┆ 14    ┆ 231.128571 ┆ 223.0      ┆ 288.0      │
│ 10136255    ┆ 3     ┆ 221.75     ┆ 157.399994 ┆ 349.649994 │
│ 7917605     ┆ 5     ┆ 125.599998 ┆ 111.5      ┆ 176.0      │
│ 7186708     ┆ 3     ┆ 253.199997 ┆ 150.0      ┆ 455.0      │
│ 4542306     ┆ 14    ┆ 89.222862  ┆ 56

In [170]:
ht_wt_df.collect().sort('PATIENT_NUM').filter(pl.col('PATIENT_NUM') == 4542306          )['NVAL_NUM'].to_list()

[69.5999984741211,
 69.19999694824219,
 155.0,
 151.1199951171875,
 125.0,
 139.0,
 125.0]

#### upon manual examination weight below 65lbs and 600 lbs

In [167]:
ht_wt_df = ht_wt_df.filter((pl.col('NVAL_NUM') > 65) & (pl.col('NVAL_NUM') < 600))

In [171]:
### transforming to modeling dataset for bmi
average_weight_per_patient = ht_wt_df.group_by("PATIENT_NUM").agg([
    pl.col("NVAL_NUM").median().alias("median_weight")
])

In [172]:
average_weight_per_patient.collect()

PATIENT_NUM,median_weight
i64,f32
4544682,144.200012
6259614,258.125
1986800,105.0
4606955,128.0
4005165,341.0
…,…
3446206,142.399994
2666157,135.300003
3535739,195.899994


### Height

In [173]:
height_df.head().collect()

PATIENT_NUM,CONCEPT_PATH,NAME_CHAR,NVAL_NUM,UNITS_CD
i64,str,str,"decimal[18,5]",str
5375663,"""\i2b2\Vitals\Height\> 84 inche…","""Vitals height more than 84 inc…",84.96000,"""In"""
1982351,"""\i2b2\Vitals\Height\> 84 inche…","""Vitals height more than 84 inc…",91.00000,"""In"""
2784015,"""\i2b2\Vitals\Height\> 84 inche…","""Vitals height more than 84 inc…",129.00000,"""In"""
4814083,"""\i2b2\Vitals\Height\> 84 inche…","""Vitals height more than 84 inc…",123.00000,"""In"""
6783073,"""\i2b2\Vitals\Height\> 84 inche…","""Vitals height more than 84 inc…",186.96000,"""In"""


In [174]:
print("Number of unique patients: ",len(np.unique(height_df.select('PATIENT_NUM').collect()['PATIENT_NUM'].to_list())))

Number of unique patients:  949537


In [175]:
print("Number of records: ",height_df.collect().shape)

Number of records:  (9506009, 5)


In [176]:
print("Number of unique BMI CONCEPT_CD ranges : ", np.unique(height_df.select('CONCEPT_PATH').collect()['CONCEPT_PATH'].to_list()))

Number of unique BMI CONCEPT_CD ranges :  ['\\i2b2\\Vitals\\Height\\> 84 inches\\'
 '\\i2b2\\Vitals\\Height\\[0-84 inches]\\']


In [177]:
height_df.sort('PATIENT_NUM').filter(#(pl.col('NVAL_NUM')< 96) &
                                    (pl.col('NVAL_NUM') > 84)).collect()

PATIENT_NUM,CONCEPT_PATH,NAME_CHAR,NVAL_NUM,UNITS_CD
i64,str,str,"decimal[18,5]",str
4365,"""\i2b2\Vitals\Height\> 84 inche…","""Vitals height more than 84 inc…",165.10000,"""In"""
4365,"""\i2b2\Vitals\Height\> 84 inche…","""Vitals height more than 84 inc…",165.10000,"""In"""
4365,"""\i2b2\Vitals\Height\> 84 inche…","""Vitals height more than 84 inc…",165.10000,"""In"""
4365,"""\i2b2\Vitals\Height\> 84 inche…","""Vitals height more than 84 inc…",165.10000,"""In"""
4365,"""\i2b2\Vitals\Height\> 84 inche…","""Vitals height more than 84 inc…",165.10000,"""In"""
…,…,…,…,…
12381942,"""\i2b2\Vitals\Height\> 84 inche…","""Vitals height more than 84 inc…",185.42000,"""In"""
12381942,"""\i2b2\Vitals\Height\> 84 inche…","""Vitals height more than 84 inc…",185.42000,"""In"""
12381942,"""\i2b2\Vitals\Height\> 84 inche…","""Vitals height more than 84 inc…",185.42000,"""In"""


In [178]:
height_df.filter(pl.col('PATIENT_NUM') == 10270897).sort('PATIENT_NUM').collect()

PATIENT_NUM,CONCEPT_PATH,NAME_CHAR,NVAL_NUM,UNITS_CD
i64,str,str,"decimal[18,5]",str


In [179]:
def custom_height_aggregator(ht_list):
    ht_list = [ht for ht in ht_list if (ht>36 and ht< 96)]
    if ht_list == []:
        return None
    return max(set(ht_list), key=ht_list.count)
    

In [180]:
mode_height_df = height_df.group_by("PATIENT_NUM").agg([
    pl.col("NVAL_NUM").map_elements(custom_height_aggregator, return_dtype = pl.Float32).alias("mode_height")
])

In [181]:
mode_height_df.collect()

PATIENT_NUM,mode_height
i64,f32
6113971,69.0
2891514,67.0
1433635,65.5
7596365,59.0
1062802,62.0
…,…
1808237,64.0
4827524,64.0
10943064,63.0


In [183]:
print("Number of unique patients in the mode  height df", len(np.unique(mode_height_df.select('PATIENT_NUM').collect()['PATIENT_NUM'].to_list())))

Number of unique patients in the mode  height df 949537


## Calculating BMI

In [186]:
# Join the DataFrames on 'patient_id'
df_combined = mode_height_df.join(average_weight_per_patient, on="PATIENT_NUM", how="inner")

# Calculate BMI
df_combined = df_combined.with_columns(
    (pl.col("median_weight") / (pl.col("mode_height") ** 2) * 703).alias("BMI")
)

In [187]:
df_combined.collect()

PATIENT_NUM,mode_height,median_weight,BMI
i64,f32,f32,f64
4233642,63.0,95.0,16.826657
3885877,63.0,111.0,19.660619
4045266,70.0,155.0,22.237756
7817235,67.0,206.0,32.260638
1896094,67.0,209.199997,32.761774
…,…,…,…
283122,72.0,207.300003,28.111865
2852681,62.0,189.0,34.564775
4594992,62.0,159.0,29.078305


In [189]:
df_combined.filter(pl.col('BMI')> 30).collect()

PATIENT_NUM,mode_height,median_weight,BMI
i64,f32,f32,f64
3224970,61.0,166.0,31.361999
6757193,67.0,288.5,45.180551
9111503,72.0,233.0,31.59703
3104893,61.0,160.154999,30.257717
1300720,69.0,246.699997,36.427241
…,…,…,…
2479633,62.0,200.5,36.667924
6546243,65.5,188.0,30.805665
5958348,66.75,256.0,40.391758


In [190]:
df_combined.collect().write_parquet('./Determine_cohort_files/Determine_modeling_bmi_data_manual.parquet')

### BP

In [191]:
### filtering Diastolic BP data
dia_bp_df = bmi_bP_df.filter(pl.col('CONCEPT_CD').str.contains('VIT\|DIA'))
sys_bp_df = bmi_bP_df.filter(pl.col('CONCEPT_CD').str.contains('VIT\|SYS'))

In [211]:
dia_bp_df = bmi_bP_df.with_columns(pl.col('NVAL_NUM').cast(pl.Float32).alias('NVAL_NUM'))
sys_bp_df = bmi_bP_df.with_columns(pl.col('NVAL_NUM').cast(pl.Float32).alias('NVAL_NUM'))

In [212]:
dia_bp_df.collect().head()

PATIENT_NUM,CONCEPT_CD,NVAL_NUM,UNITS_CD,START_DATE
i64,str,f32,str,datetime[μs]
6735102,"""VIT|WT:0-499""",139.0,"""Lb""",2017-03-16 00:00:00
3198040,"""VIT|WT:0-499""",168.0,"""Lb""",2017-01-16 00:00:00
1362288,"""VIT|WT:0-499""",134.800003,"""Lb""",2017-02-15 00:00:00
11852072,"""VIT|WT:0-499""",223.0,"""Lb""",2017-02-16 00:00:00
3598189,"""VIT|WT:0-499""",208.0,"""Lb""",2017-02-13 00:00:00


In [213]:
print("Number of unique patients in diastolic bp: ",len(np.unique(dia_bp_df.select('PATIENT_NUM').collect()['PATIENT_NUM'].to_list())))
print("Number of unique patients in systolic bp: ",len(np.unique(sys_bp_df.select('PATIENT_NUM').collect()['PATIENT_NUM'].to_list())))

Number of unique patients in diastolic bp:  680089
Number of unique patients in systolic bp:  680089


In [214]:
print("Number of records in dia bp: ",dia_bp_df.collect().shape)
print("Number of records in sys bp: ",sys_bp_df.collect().shape)

Number of records in dia bp:  (9095625, 5)
Number of records in sys bp:  (9095625, 5)


In [215]:
print("Number of unique dia BP CONCEPT_CD values : ", np.unique(dia_bp_df.select('CONCEPT_CD').collect()['CONCEPT_CD'].to_list(), return_counts =True))
print("Number of unique sys BP CONCEPT_CD values : ", np.unique(sys_bp_df.select('CONCEPT_CD').collect()['CONCEPT_CD'].to_list(), return_counts =True))

Number of unique dia BP CONCEPT_CD values :  (array(['VIT|BMI:0-79', 'VIT|BMI:>79', 'VIT|DIASTOLIC:0-200',
       'VIT|DIASTOLIC:>200', 'VIT|SYSTOLIC:0-200', 'VIT|SYSTOLIC:>200',
       'VIT|WT:0-499', 'VIT|WT:>499'], dtype='<U19'), array([2105400,    2452, 2405834,      82, 2402266,    3655, 2175386,
           550]))
Number of unique sys BP CONCEPT_CD values :  (array(['VIT|BMI:0-79', 'VIT|BMI:>79', 'VIT|DIASTOLIC:0-200',
       'VIT|DIASTOLIC:>200', 'VIT|SYSTOLIC:0-200', 'VIT|SYSTOLIC:>200',
       'VIT|WT:0-499', 'VIT|WT:>499'], dtype='<U19'), array([2105400,    2452, 2405834,      82, 2402266,    3655, 2175386,
           550]))


In [216]:
print("Number of unique measuring units in dia bp : ", np.unique(dia_bp_df.select('UNITS_CD').collect()['UNITS_CD'].to_list()))
print("Number of unique measuring units in sys bp : ", np.unique(sys_bp_df.select('UNITS_CD').collect()['UNITS_CD'].to_list()))

Number of unique measuring units in dia bp :  ['In' 'Lb' 'kg/m2' 'mmHg']
Number of unique measuring units in sys bp :  ['In' 'Lb' 'kg/m2' 'mmHg']


In [217]:
dia_list = dia_bp_df.select('NVAL_NUM').collect()['NVAL_NUM'].to_list()
sys_list = sys_bp_df.select('NVAL_NUM').collect()['NVAL_NUM'].to_list()

print("Min and max of dia bp: ", min(dia_list), max(dia_list))
print("Min and max of sys bp: ", min(sys_list), max(sys_list))

Min and max of dia bp:  0.0 280222.21875
Min and max of sys bp:  0.0 280222.21875


In [218]:
dia_bp_df.filter(pl.col('NVAL_NUM') > 120).collect()

PATIENT_NUM,CONCEPT_CD,NVAL_NUM,UNITS_CD,START_DATE
i64,str,f32,str,datetime[μs]
6735102,"""VIT|WT:0-499""",139.0,"""Lb""",2017-03-16 00:00:00
3198040,"""VIT|WT:0-499""",168.0,"""Lb""",2017-01-16 00:00:00
1362288,"""VIT|WT:0-499""",134.800003,"""Lb""",2017-02-15 00:00:00
11852072,"""VIT|WT:0-499""",223.0,"""Lb""",2017-02-16 00:00:00
3598189,"""VIT|WT:0-499""",208.0,"""Lb""",2017-02-13 00:00:00
…,…,…,…,…
254859,"""VIT|WT:0-499""",161.0,"""Lb""",2017-01-16 00:00:00
510848,"""VIT|WT:0-499""",129.399994,"""Lb""",2017-01-09 00:00:00
3912363,"""VIT|WT:0-499""",294.0,"""Lb""",2017-03-10 00:00:00


#### for diastolic values rnage to be considered 40 and 160 
#### for systolic values range to be condsidered 60 and 190

In [219]:
sys_bp_df.filter(pl.col('PATIENT_NUM') == 5023457).collect()

PATIENT_NUM,CONCEPT_CD,NVAL_NUM,UNITS_CD,START_DATE
i64,str,f32,str,datetime[μs]
5023457,"""VIT|SYSTOLIC:0-200""",140.0,"""mmHg""",2017-01-23 00:00:00
5023457,"""VIT|DIASTOLIC:0-200""",90.0,"""In""",2017-01-23 00:00:00


In [220]:
dia_bp_df  = dia_bp_df.filter((pl.col('NVAL_NUM') > 40) &
                             (pl.col('NVAL_NUM') < 160))
sys_bp_df  = sys_bp_df.filter((pl.col('NVAL_NUM') > 60) &
                             (pl.col('NVAL_NUM') < 190))

In [221]:
print("Number of unique pateint ids in diasytolic data: ",len(np.unique(dia_bp_df.select('PATIENT_NUM').collect()['PATIENT_NUM'].to_list())))
print("Number of unique pateint ids in systolic data: ",len(np.unique(sys_bp_df.select('PATIENT_NUM').collect()['PATIENT_NUM'].to_list())))

Number of unique pateint ids in diasytolic data:  676081
Number of unique pateint ids in systolic data:  677752


In [223]:
### transforming to modeling dataset for bmi
average_dia_bp_per_patient = dia_bp_df.group_by("PATIENT_NUM").agg([
    pl.col("NVAL_NUM").mean().alias("median_diastolic_value"),
#     pl.col("NVAL_NUM").min().alias("min_diastolic_value"),
#     pl.col("NVAL_NUM").max().alias("max_diastolic_value")
])

average_sys_bp_per_patient = sys_bp_df.group_by("PATIENT_NUM").agg([
    pl.col("NVAL_NUM").mean().alias("median_systolic_value"),
#     pl.col("NVAL_NUM").min().alias("min_systolic_value"),
#     pl.col("NVAL_NUM").max().alias("max_systolic_value")
])

In [225]:
average_dia_bp_per_patient.collect().write_parquet('./Determine_cohort_files/Determine_modeling_diastolic_bp_data.parquet')
average_sys_bp_per_patient.collect().write_parquet('./Determine_cohort_files/Determine_modeling_systolic_bp_data.parquet')

# Tob & smoking data

In [ ]:
tob_df.head().collect()

In [ ]:
tob_df.filter(pl.col('PATIENT_NUM')==722088).collect().head(10)

In [ ]:
tob_df = tob_df.with_columns(pl.lit(1).alias('Indicator'))

In [ ]:
tob_df.head().collect()

# Community vital signs

In [226]:
community_vital_df.collect().head()

PATIENT_NUM,ZIP_CD,Indicator,PERIOD,FACT
i64,str,str,str,str
5627352,"""01001""","""ACS_GINI""","""2016-2020""","""41.85"""
5621397,"""01001""","""ACS_GINI""","""2016-2020""","""41.85"""
563554,"""01001""","""ACS_GINI""","""2016-2020""","""41.85"""
528640,"""01001""","""ACS_GINI""","""2016-2020""","""41.85"""
517732,"""01001""","""ACS_GINI""","""2016-2020""","""41.85"""


In [227]:
community_vital_df = community_vital_df.with_columns(pl.col('FACT').cast(pl.Float32).alias('FACT'))

In [228]:
community_vital_df.null_count().collect()

PATIENT_NUM,ZIP_CD,Indicator,PERIOD,FACT
u32,u32,u32,u32,u32
0,0,0,0,6836


In [229]:
community_vital_df.filter(pl.col('PATIENT_NUM')==556).collect()

PATIENT_NUM,ZIP_CD,Indicator,PERIOD,FACT
i64,str,str,str,f32
556,"""95519""","""ACS_GINI""","""2016-2020""",45.75
556,"""95519""","""ACS_MedHHIncome""","""2016-2020""",56422.0
556,"""95519""","""ACS_pctCollGrad""","""2016-2020""",33.120899
556,"""95519""","""ACS_pctPoverty100""","""2016-2020""",16.0502
556,"""95519""","""ACS_Unemployment""","""2016-2020""",3.09941


In [230]:
community_vital_df.collect().shape

(5235440, 5)

In [231]:
len(community_vital_df.unique(subset=["PATIENT_NUM", "ZIP_CD"]).collect())

1047088

# SDoH screening data

In [41]:
sdoh_screening_df = sdoh_screening_df.with_columns(pl.col('QUANTITY_NUM').cast(pl.Float32).alias('QUANTITY_NUM'))

In [42]:
sdoh_screening_df.head()

PATIENT_NUM,CONCEPT_PATH,NAME_CHAR,CONCEPT_CD,QUANTITY_NUM
i64,str,str,str,f32
6833020,"""\i2b2\SDH\Food Insecurity\ADV0…","""Did your household run out of …","""SDH:ADV0041 """,0.0
2240700,"""\i2b2\SDH\Food Insecurity\ADV0…","""Did your household run out of …","""SDH:ADV0041 """,0.0
2790074,"""\i2b2\SDH\Food Insecurity\ADV0…","""Did your household run out of …","""SDH:ADV0041 """,0.0
2116451,"""\i2b2\SDH\Food Insecurity\ADV0…","""Did your household run out of …","""SDH:ADV0041 """,0.0
4102386,"""\i2b2\SDH\Food Insecurity\ADV0…","""Did your household run out of …","""SDH:ADV0041 """,0.0


In [44]:
print("Number of unique patients in SDoH screening data",len(np.unique(sdoh_screening_df['PATIENT_NUM'].to_list())))
print("Number of unique answers in SDoH screening data",len(np.unique(sdoh_screening_df['QUANTITY_NUM'].to_list())))
print("Number of unique features in SDoH screening data",len(np.unique(sdoh_screening_df['CONCEPT_CD'].to_list())))

Number of unique patients in SDoH screening data 11447
Number of unique answers in SDoH screening data 3
Number of unique features in SDoH screening data 50


In [48]:
np.unique(sdoh_screening_df['QUANTITY_NUM'].to_list())

array([0., 1., 9.])

In [49]:
sdoh_screening_pivot_df = sdoh_screening_df.pivot(on = 'CONCEPT_CD', index = 'PATIENT_NUM', values = 'QUANTITY_NUM', aggregate_function = 'max').fill_null(9)
sdoh_screening_pivot_df.head()

PATIENT_NUM,SDH:ADV0041,SDH:ADV0046,SDH:ADV0051,SDH:ADV0042,SDH:ADV0047,SDH:ADV0048,SDH:ADV0052,SDH:ADV0050,SDH:ADV0055,SDH:ADV0049,SDH:ADV0043,SDH:ADV0054,SDH:ADV0053,SDH:ADV0056,SDH:ADV0070,SDH:ADV0071,SDH:ADV0044,SDH:ADV0078,SDH:ADV0073,SDH:ADV0086,SDH:ADV0093,SDH:ADV0095,SDH:ADV0067,SDH:ADV0074,SDH:ADV0075,SDH:ADV0076,SDH:ADV0088,SDH:ADV0069,SDH:ADV0072,SDH:ADV0068,SDH:ADV0089,SDH:ADV0077,SDH:ADV0079,SDH:ADV0080,SDH:ADV0085,SDH:ADV0081,SDH:ADV0082,SDH:ADV0087,SDH:ADV0083,SDH:ADV0084,SDH:ADV0090,SDH:ADV0091,SDH:ADV0092,SDH:ADV0094,SDH:ADV0247,SDH:ADV0248,SDH:ADV0249,SDH:ADV0250,SDH:ADV0400,SDH:ADV0401
i64,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32
6833020,0.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,0.0,9.0
2240700,0.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,0.0,9.0
2790074,0.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,0.0,9.0
2116451,0.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,0.0,9.0
4102386,0.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,0.0,9.0


In [55]:
## Create dummy variable:
sdoh_screening_pivot_df_dummies = sdoh_screening_pivot_df.select(sdoh_screening_pivot_df.columns[1:]).to_dummies()
sdoh_screening_pivot_df_dummies

SDH:ADV0041 _0.0,SDH:ADV0041 _1.0,SDH:ADV0041 _9.0,SDH:ADV0046 _0.0,SDH:ADV0046 _1.0,SDH:ADV0046 _9.0,SDH:ADV0051 _0.0,SDH:ADV0051 _1.0,SDH:ADV0051 _9.0,SDH:ADV0042 _9.0,SDH:ADV0047 _0.0,SDH:ADV0047 _1.0,SDH:ADV0047 _9.0,SDH:ADV0048 _0.0,SDH:ADV0048 _1.0,SDH:ADV0048 _9.0,SDH:ADV0052 _0.0,SDH:ADV0052 _1.0,SDH:ADV0052 _9.0,SDH:ADV0050 _0.0,SDH:ADV0050 _1.0,SDH:ADV0050 _9.0,SDH:ADV0055 _9.0,SDH:ADV0049 _0.0,SDH:ADV0049 _1.0,SDH:ADV0049 _9.0,SDH:ADV0043 _0.0,SDH:ADV0043 _1.0,SDH:ADV0043 _9.0,SDH:ADV0054 _0.0,SDH:ADV0054 _1.0,SDH:ADV0054 _9.0,SDH:ADV0053 _0.0,SDH:ADV0053 _1.0,SDH:ADV0053 _9.0,SDH:ADV0056 _9.0,SDH:ADV0070 _0.0,…,SDH:ADV0069 _0.0,SDH:ADV0069 _1.0,SDH:ADV0069 _9.0,SDH:ADV0072 _0.0,SDH:ADV0072 _1.0,SDH:ADV0072 _9.0,SDH:ADV0068 _9.0,SDH:ADV0089 _0.0,SDH:ADV0089 _1.0,SDH:ADV0089 _9.0,SDH:ADV0077 _9.0,SDH:ADV0079 _9.0,SDH:ADV0080 _9.0,SDH:ADV0085 _9.0,SDH:ADV0081 _9.0,SDH:ADV0082 _9.0,SDH:ADV0087 _9.0,SDH:ADV0083 _0.0,SDH:ADV0083 _1.0,SDH:ADV0083 _9.0,SDH:ADV0084 _0.0,SDH:ADV0084 _1.0,SDH:ADV0084 _9.0,SDH:ADV0090 _9.0,SDH:ADV0091 _9.0,SDH:ADV0092 _9.0,SDH:ADV0094 _9.0,SDH:ADV0247 _9.0,SDH:ADV0248 _9.0,SDH:ADV0249 _9.0,SDH:ADV0250 _9.0,SDH:ADV0400 _0.0,SDH:ADV0400 _1.0,SDH:ADV0400 _9.0,SDH:ADV0401 _0.0,SDH:ADV0401 _1.0,SDH:ADV0401 _9.0
u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,…,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8
1,0,0,0,0,1,0,0,1,1,0,0,1,0,0,1,0,0,1,0,0,1,1,0,0,1,0,0,1,0,0,1,0,0,1,1,0,…,0,0,1,0,0,1,1,0,0,1,1,1,1,1,1,1,1,0,0,1,0,0,1,1,1,1,1,1,1,1,1,1,0,0,0,0,1
1,0,0,0,0,1,0,0,1,1,0,0,1,0,0,1,0,0,1,0,0,1,1,0,0,1,0,0,1,0,0,1,0,0,1,1,0,…,0,0,1,0,0,1,1,0,0,1,1,1,1,1,1,1,1,0,0,1,0,0,1,1,1,1,1,1,1,1,1,1,0,0,0,0,1
1,0,0,0,0,1,0,0,1,1,0,0,1,0,0,1,0,0,1,0,0,1,1,0,0,1,0,0,1,0,0,1,0,0,1,1,0,…,0,0,1,0,0,1,1,0,0,1,1,1,1,1,1,1,1,0,0,1,0,0,1,1,1,1,1,1,1,1,1,1,0,0,0,0,1
1,0,0,0,0,1,0,0,1,1,0,0,1,0,0,1,0,0,1,0,0,1,1,0,0,1,0,0,1,0,0,1,0,0,1,1,0,…,0,0,1,0,0,1,1,0,0,1,1,1,1,1,1,1,1,0,0,1,0,0,1,1,1,1,1,1,1,1,1,1,0,0,0,0,1
1,0,0,0,0,1,0,0,1,1,0,0,1,0,0,1,0,0,1,0,0,1,1,0,0,1,0,0,1,0,0,1,0,0,1,1,0,…,0,0,1,0,0,1,1,0,0,1,1,1,1,1,1,1,1,0,0,1,0,0,1,1,1,1,1,1,1,1,1,1,0,0,0,0,1
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
0,0,1,0,0,1,0,0,1,1,0,0,1,0,0,1,0,0,1,0,0,1,1,0,0,1,0,0,1,0,0,1,0,0,1,1,0,…,0,0,1,0,0,1,1,0,0,1,1,1,1,1,1,1,1,0,0,1,0,0,1,1,1,1,1,1,1,1,1,1,0,0,0,0,1
0,0,1,0,0,1,0,0,1,1,0,0,1,0,0,1,0,0,1,0,0,1,1,0,0,1,0,0,1,0,0,1,0,0,1,1,0,…,0,0,1,0,0,1,1,0,0,1,1,1,1,1,1,1,1,0,0,1,0,0,1,1,1,1,1,1,1,1,1,1,0,0,0,0,1
0,0,1,0,0,1,0,0,1,1,0,0,1,0,0,1,0,0,1,0,0,1,1,0,0,1,0,0,1,0,0,1,0,0,1,1,0,…,0,0,1,0,0,1,1,0,0,1,1,1,1,1,1,1,1,0,0,1,0,0,1,1,1,1,1,1,1,1,1,1,0,0,0,0,1


In [56]:
sdoh_screening_pivot_modeling_df = pl.concat([sdoh_screening_pivot_df.select('PATIENT_NUM'), sdoh_screening_pivot_df_dummies], how='horizontal')


In [61]:
req_cols = [col for col in sdoh_screening_pivot_modeling_df.columns if not col.endswith('9.0')]
req_cols

['PATIENT_NUM',
 'SDH:ADV0041   _0.0',
 'SDH:ADV0041   _1.0',
 'SDH:ADV0046   _0.0',
 'SDH:ADV0046   _1.0',
 'SDH:ADV0051   _0.0',
 'SDH:ADV0051   _1.0',
 'SDH:ADV0047   _0.0',
 'SDH:ADV0047   _1.0',
 'SDH:ADV0048   _0.0',
 'SDH:ADV0048   _1.0',
 'SDH:ADV0052   _0.0',
 'SDH:ADV0052   _1.0',
 'SDH:ADV0050   _0.0',
 'SDH:ADV0050   _1.0',
 'SDH:ADV0049   _0.0',
 'SDH:ADV0049   _1.0',
 'SDH:ADV0043   _0.0',
 'SDH:ADV0043   _1.0',
 'SDH:ADV0054   _0.0',
 'SDH:ADV0054   _1.0',
 'SDH:ADV0053   _0.0',
 'SDH:ADV0053   _1.0',
 'SDH:ADV0070   _0.0',
 'SDH:ADV0070   _1.0',
 'SDH:ADV0044   _0.0',
 'SDH:ADV0044   _1.0',
 'SDH:ADV0073   _0.0',
 'SDH:ADV0073   _1.0',
 'SDH:ADV0067   _0.0',
 'SDH:ADV0067   _1.0',
 'SDH:ADV0074   _0.0',
 'SDH:ADV0074   _1.0',
 'SDH:ADV0069   _0.0',
 'SDH:ADV0069   _1.0',
 'SDH:ADV0072   _0.0',
 'SDH:ADV0072   _1.0',
 'SDH:ADV0089   _0.0',
 'SDH:ADV0089   _1.0',
 'SDH:ADV0083   _0.0',
 'SDH:ADV0083   _1.0',
 'SDH:ADV0084   _0.0',
 'SDH:ADV0084   _1.0',
 'SDH:ADV0400   _0

In [64]:
sdoh_screening_pivot_modeling_df = sdoh_screening_pivot_modeling_df.select(req_cols)
sdoh_screening_pivot_modeling_df

PATIENT_NUM,SDH:ADV0041 _0.0,SDH:ADV0041 _1.0,SDH:ADV0046 _0.0,SDH:ADV0046 _1.0,SDH:ADV0051 _0.0,SDH:ADV0051 _1.0,SDH:ADV0047 _0.0,SDH:ADV0047 _1.0,SDH:ADV0048 _0.0,SDH:ADV0048 _1.0,SDH:ADV0052 _0.0,SDH:ADV0052 _1.0,SDH:ADV0050 _0.0,SDH:ADV0050 _1.0,SDH:ADV0049 _0.0,SDH:ADV0049 _1.0,SDH:ADV0043 _0.0,SDH:ADV0043 _1.0,SDH:ADV0054 _0.0,SDH:ADV0054 _1.0,SDH:ADV0053 _0.0,SDH:ADV0053 _1.0,SDH:ADV0070 _0.0,SDH:ADV0070 _1.0,SDH:ADV0044 _0.0,SDH:ADV0044 _1.0,SDH:ADV0073 _0.0,SDH:ADV0073 _1.0,SDH:ADV0067 _0.0,SDH:ADV0067 _1.0,SDH:ADV0074 _0.0,SDH:ADV0074 _1.0,SDH:ADV0069 _0.0,SDH:ADV0069 _1.0,SDH:ADV0072 _0.0,SDH:ADV0072 _1.0,SDH:ADV0089 _0.0,SDH:ADV0089 _1.0,SDH:ADV0083 _0.0,SDH:ADV0083 _1.0,SDH:ADV0084 _0.0,SDH:ADV0084 _1.0,SDH:ADV0400 _0.0,SDH:ADV0400 _1.0,SDH:ADV0401 _0.0,SDH:ADV0401 _1.0
i64,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8
6833020,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
2240700,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
2790074,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
2116451,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
4102386,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
3981388,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
5537882,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
709541,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0


In [65]:
sdoh_screening_pivot_modeling_df.write_parquet('./Determine_modeling_datasets/Determine_sdoh_screening_pivot.parquet')

# Pivoting dataframes for modeling

In [256]:
class pivoting_config:
    min_association_patients = 50
    diag_phemap_pivot = True
    diag_icd_nchars_pivot = True


## Medications pivot

In [232]:
# Loading the dataset
medications_df = pl.read_parquet("./Determine_cohort_files/Determine_cohort_medications_active_ing.parquet")

In [233]:
medications_df = medications_df.with_columns(pl.lit(1).alias('usage'))

In [234]:
medications_df.head()

PATIENT_NUM,ENCOUNTER_NUM,START_DATE,RxNorm,Medication_Name,RX_StartDate,RX_Dose_Ordered,Rx_Dose_Ordered_Unit,RX_Quantity,Active_ingrident,usage
i64,i64,datetime[μs],str,str,datetime[μs],"decimal[15,5]",str,"decimal[15,8]",str,i32
556,25455227,2016-02-12 00:00:00,"""RXNORM:1094104 ""","""Phenazopyridine Hydrochloride …",2016-02-12 00:00:00,200.00000,"""mg""",10.00000000,"""phenazopyridine""",1
556,25455227,2016-02-12 00:00:00,"""RXNORM:1648755 ""","""Nitrofurantoin, Macrocrystals …",2016-02-12 00:00:00,100.00000,"""mg""",14.00000000,"""nitrofurantoin""",1
556,26227551,2016-02-01 00:00:00,"""RXNORM:197886 ""","""Hydrochlorothiazide 12.5 Mg / …",2016-02-01 00:00:00,null,"""NI""",30.00000000,"""lisinopril_hydrochlorothiazide""",1
556,27284087,2016-05-03 00:00:00,"""RXNORM:197886 ""","""Hydrochlorothiazide 12.5 Mg / …",2016-05-03 00:00:00,null,"""NI""",30.00000000,"""lisinopril_hydrochlorothiazide""",1
556,27284087,2016-05-03 00:00:00,"""RXNORM:197361 ""","""Amlodipine 5 Mg Oral Tablet (1…",2016-05-03 00:00:00,5.00000,"""mg""",30.00000000,"""amlodipine""",1


In [235]:
### Pivoting
medications_df_pivot = medications_df.pivot(on = 'Active_ingrident', index = 'PATIENT_NUM', values = 'usage', aggregate_function = 'max')

In [236]:
medications_df_pivot.head()

PATIENT_NUM,phenazopyridine,nitrofurantoin,lisinopril_hydrochlorothiazide,amlodipine,cephalexin,albuterol,nystatin,metronidazole,mometasone_formoterol,varicella-zoster virus vaccine live (Oka-Merck) strain,potassium chloride,chlorthalidone,ketorolac,azithromycin,triamcinolone,gabapentin,acetaminophen_hydrocodone,capsaicin,naproxen,tramadol,ibuprofen,codeine_promethazine,omeprazole,clotrimazole,lorazepam,fluticasone,calcium carbonate,loratadine,paroxetine,cholecalciferol,aluminum chloride,penicillin G,sulfamethoxazole_trimethoprim,ondansetron,doxylamine_pyridoxine,folic acid,…,chlorpheniramine_codeine_pseudoephedrine,nabilone,zinc acetate,vitamin B12_ascorbic acid_iron carbonyl_folic acid_docusate,thioguanine,carbinoxamine,ketamine,rx_cui:1303740,ammonia,tyrosine,sodium phosphate,rx_cui:1150087,factor VIII,ethanol,fingolimod,rx_cui:1247404,icatibant,rx_cui:1356525,polysaccharide iron complex_vitamin B12_folic acid_heme iron polypeptide,rx_cui:856892,rx_cui:313409,"poliovirus vaccine inactivated, type 1 (Mahoney)_poliovirus vaccine inactivated, type 2 (MEF-1)_poliovirus vaccine inactivated, type 3 (Saukett)_acellular pertussis vaccine, inactivated_diphtheria toxoid vaccine, inactivated_tetanus toxoid vaccine, inactivated",rx_cui:198725,rx_cui:1249797,rx_cui:966545,dexbrompheniramine_pseudoephedrine,benzethonium_benzocaine,rx_cui:237778,rx_cui:2174531,rx_cui:204969,rx_cui:310468,ganirelix,follicle stimulating hormone_luteinizing hormone,aztreonam,isoniazid_rifampin,lactate_calcium chloride_glucose_potassium chloride_sodium chloride,glucose_potassium chloride_sodium chloride
i64,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,…,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32
556,1,1,1,1,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
560,null,null,null,null,1,1,1,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
561,null,null,null,null,null,null,null,1,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
562,null,null,null,null,null,null,null,null,1,1,1,1,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
564,null,null,null,null,null,1,null,null,null,null,null,null,1,1,1,1,1,1,1,1,1,1,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null


In [237]:
medications_df_pivot = medications_df_pivot.fill_null(0)

In [238]:
medications_df_pivot.head()

PATIENT_NUM,phenazopyridine,nitrofurantoin,lisinopril_hydrochlorothiazide,amlodipine,cephalexin,albuterol,nystatin,metronidazole,mometasone_formoterol,varicella-zoster virus vaccine live (Oka-Merck) strain,potassium chloride,chlorthalidone,ketorolac,azithromycin,triamcinolone,gabapentin,acetaminophen_hydrocodone,capsaicin,naproxen,tramadol,ibuprofen,codeine_promethazine,omeprazole,clotrimazole,lorazepam,fluticasone,calcium carbonate,loratadine,paroxetine,cholecalciferol,aluminum chloride,penicillin G,sulfamethoxazole_trimethoprim,ondansetron,doxylamine_pyridoxine,folic acid,…,chlorpheniramine_codeine_pseudoephedrine,nabilone,zinc acetate,vitamin B12_ascorbic acid_iron carbonyl_folic acid_docusate,thioguanine,carbinoxamine,ketamine,rx_cui:1303740,ammonia,tyrosine,sodium phosphate,rx_cui:1150087,factor VIII,ethanol,fingolimod,rx_cui:1247404,icatibant,rx_cui:1356525,polysaccharide iron complex_vitamin B12_folic acid_heme iron polypeptide,rx_cui:856892,rx_cui:313409,"poliovirus vaccine inactivated, type 1 (Mahoney)_poliovirus vaccine inactivated, type 2 (MEF-1)_poliovirus vaccine inactivated, type 3 (Saukett)_acellular pertussis vaccine, inactivated_diphtheria toxoid vaccine, inactivated_tetanus toxoid vaccine, inactivated",rx_cui:198725,rx_cui:1249797,rx_cui:966545,dexbrompheniramine_pseudoephedrine,benzethonium_benzocaine,rx_cui:237778,rx_cui:2174531,rx_cui:204969,rx_cui:310468,ganirelix,follicle stimulating hormone_luteinizing hormone,aztreonam,isoniazid_rifampin,lactate_calcium chloride_glucose_potassium chloride_sodium chloride,glucose_potassium chloride_sodium chloride
i64,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,…,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32
556,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
560,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
561,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
562,0,0,0,0,0,0,0,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
564,0,0,0,0,0,1,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [239]:
medications_df_pivot.null_count().sum()

PATIENT_NUM,phenazopyridine,nitrofurantoin,lisinopril_hydrochlorothiazide,amlodipine,cephalexin,albuterol,nystatin,metronidazole,mometasone_formoterol,varicella-zoster virus vaccine live (Oka-Merck) strain,potassium chloride,chlorthalidone,ketorolac,azithromycin,triamcinolone,gabapentin,acetaminophen_hydrocodone,capsaicin,naproxen,tramadol,ibuprofen,codeine_promethazine,omeprazole,clotrimazole,lorazepam,fluticasone,calcium carbonate,loratadine,paroxetine,cholecalciferol,aluminum chloride,penicillin G,sulfamethoxazole_trimethoprim,ondansetron,doxylamine_pyridoxine,folic acid,…,chlorpheniramine_codeine_pseudoephedrine,nabilone,zinc acetate,vitamin B12_ascorbic acid_iron carbonyl_folic acid_docusate,thioguanine,carbinoxamine,ketamine,rx_cui:1303740,ammonia,tyrosine,sodium phosphate,rx_cui:1150087,factor VIII,ethanol,fingolimod,rx_cui:1247404,icatibant,rx_cui:1356525,polysaccharide iron complex_vitamin B12_folic acid_heme iron polypeptide,rx_cui:856892,rx_cui:313409,"poliovirus vaccine inactivated, type 1 (Mahoney)_poliovirus vaccine inactivated, type 2 (MEF-1)_poliovirus vaccine inactivated, type 3 (Saukett)_acellular pertussis vaccine, inactivated_diphtheria toxoid vaccine, inactivated_tetanus toxoid vaccine, inactivated",rx_cui:198725,rx_cui:1249797,rx_cui:966545,dexbrompheniramine_pseudoephedrine,benzethonium_benzocaine,rx_cui:237778,rx_cui:2174531,rx_cui:204969,rx_cui:310468,ganirelix,follicle stimulating hormone_luteinizing hormone,aztreonam,isoniazid_rifampin,lactate_calcium chloride_glucose_potassium chloride_sodium chloride,glucose_potassium chloride_sodium chloride
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,…,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [240]:
medications_df_pivot.write_parquet("./Determine_Cohort_files_pivot/Determine_medications_active_ingredients_pivot.parquet")

## Lab results pivot

In [248]:
lab_results_path = './Determine_cohort_files/Determine_cohort_labresults_domain_expert_median.parquet'
lab_results_df = pl.read_parquet(lab_results_path)
lab_results_df.head()

PATIENT_NUM,LabLOINC,Result_Number
i64,str,f32
469479,"""LOINC:704-7""",0.028
5906095,"""LOINC:43396-1""",194.0
555871,"""LOINC:10834-0""",3.0
5468509,"""LOINC:43396-1""",117.0
6318519,"""LOINC:48643-1""",140.5


In [249]:
lab_results_df.null_count()

PATIENT_NUM,LabLOINC,Result_Number
u32,u32,u32
0,0,0


In [250]:
lab_results_pivot_df = lab_results_df.pivot(on = 'LabLOINC', index = 'PATIENT_NUM', values = 'Result_Number')

In [251]:
lab_results_pivot_df.null_count()

PATIENT_NUM,LOINC:704-7,LOINC:43396-1,LOINC:10834-0,LOINC:48643-1,LOINC:788-0,LOINC:11579-0,LOINC:17861-6,LOINC:2085-9,LOINC:4544-3,LOINC:2160-0,LOINC:713-8,LOINC:751-8,LOINC:718-7,LOINC:777-3,LOINC:1742-6,LOINC:2075-0,LOINC:30428-7,LOINC:2885-2,LOINC:2571-8,LOINC:1759-0,LOINC:3094-0,LOINC:792-2,LOINC:1920-8,LOINC:33914-3,LOINC:5905-5,LOINC:785-6,LOINC:1968-7,LOINC:2345-7,LOINC:2823-3,LOINC:51584-1,LOINC:62291-0,LOINC:3040-3,LOINC:3097-3,LOINC:1751-7,LOINC:2857-1,LOINC:2093-3,…,LOINC:30565-6,LOINC:26524-9,LOINC:13797-6,LOINC:69573-4,LOINC:38190-5,LOINC:4621-9,LOINC:83083-6,LOINC:2705-2,LOINC:26610-6,LOINC:19176-7,LOINC:9616-4,LOINC:30542-5,LOINC:13706-7,LOINC:33805-3,LOINC:26452-3,LOINC:14044-2,LOINC:13518-6,LOINC:33254-4,LOINC:9410-2,LOINC:19074-4,LOINC:56448-4,LOINC:30328-9,LOINC:13526-9,LOINC:46737-3,LOINC:18314-5,LOINC:14328-9,LOINC:12480-0,LOINC:74428-4,LOINC:32173-7,LOINC:3066-8,LOINC:3096-5,LOINC:42482-0,LOINC:2744-1,LOINC:21019-5,LOINC:33325-2,LOINC:19235-1,LOINC:26881-3
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,…,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,206802,272478,204585,183008,152950,357645,146690,195681,166887,265922,210902,212327,166752,152655,186856,137167,402001,171517,201448,196451,142625,397845,171673,240677,207186,171929,394756,143316,146187,349310,400163,398580,296497,191436,385743,188922,…,406121,406121,406121,406121,406121,406121,406121,406121,406119,406121,406121,406121,406121,406120,406120,406121,406121,406120,406121,406121,406121,406121,406121,406121,406121,406121,406121,406121,406121,406121,406121,406121,406120,406121,406121,406121,406121


In [252]:
#lab_results_pivot_df = lab_results_pivot_df.fill_null(-100)

In [253]:
lab_results_pivot_df.head()

PATIENT_NUM,LOINC:704-7,LOINC:43396-1,LOINC:10834-0,LOINC:48643-1,LOINC:788-0,LOINC:11579-0,LOINC:17861-6,LOINC:2085-9,LOINC:4544-3,LOINC:2160-0,LOINC:713-8,LOINC:751-8,LOINC:718-7,LOINC:777-3,LOINC:1742-6,LOINC:2075-0,LOINC:30428-7,LOINC:2885-2,LOINC:2571-8,LOINC:1759-0,LOINC:3094-0,LOINC:792-2,LOINC:1920-8,LOINC:33914-3,LOINC:5905-5,LOINC:785-6,LOINC:1968-7,LOINC:2345-7,LOINC:2823-3,LOINC:51584-1,LOINC:62291-0,LOINC:3040-3,LOINC:3097-3,LOINC:1751-7,LOINC:2857-1,LOINC:2093-3,…,LOINC:30565-6,LOINC:26524-9,LOINC:13797-6,LOINC:69573-4,LOINC:38190-5,LOINC:4621-9,LOINC:83083-6,LOINC:2705-2,LOINC:26610-6,LOINC:19176-7,LOINC:9616-4,LOINC:30542-5,LOINC:13706-7,LOINC:33805-3,LOINC:26452-3,LOINC:14044-2,LOINC:13518-6,LOINC:33254-4,LOINC:9410-2,LOINC:19074-4,LOINC:56448-4,LOINC:30328-9,LOINC:13526-9,LOINC:46737-3,LOINC:18314-5,LOINC:14328-9,LOINC:12480-0,LOINC:74428-4,LOINC:32173-7,LOINC:3066-8,LOINC:3096-5,LOINC:42482-0,LOINC:2744-1,LOINC:21019-5,LOINC:33325-2,LOINC:19235-1,LOINC:26881-3
i64,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,…,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32
469479,0.028,67.0,2.55,133.5,12.950001,null,10.049999,52.0,51.099998,0.885,3.0,2.542,17.200001,229.5,18.0,103.0,null,7.4,43.0,1.9,16.0,null,17.5,115.0,10.15,29.450001,null,88.5,4.65,null,null,null,null,4.85,null,119.0,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
5906095,0.04,194.0,3.1,85.5,15.1,null,10.65,51.0,42.599998,1.095,9.0,3.91,13.4,256.0,16.5,100.0,null,7.85,81.0,1.5,22.0,null,14.0,74.0,7.0,27.9,null,119.0,4.2,null,null,null,20.400002,4.75,null,245.0,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
555871,0.05,106.0,3.0,72.0,14.4,null,9.8,46.0,40.299999,1.26,3.0,7.47,13.1,302.0,8.0,104.0,null,7.6,119.0,1.6,9.0,null,12.0,62.0,6.0,29.1,null,106.0,4.1,null,null,null,6.8,4.6,null,152.0,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
5468509,null,117.0,2.7,123.0,16.200001,null,9.6,47.0,46.0,null,null,null,14.9,189.0,18.0,105.0,null,7.3,70.0,1.7,14.0,null,21.0,107.0,null,25.700001,null,96.0,5.0,null,null,null,null,4.6,null,164.0,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
6318519,0.0,null,2.25,140.5,13.5,null,9.2,null,42.599998,null,1.0,3.0,15.0,272.0,23.0,103.0,null,6.55,null,1.9,13.0,null,14.5,null,10.0,30.6,null,92.0,4.25,0.0,null,null,19.0,4.3,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null


In [254]:
if 'all_features' not in lab_results_path:
    lab_results_pivot_df.write_parquet("./Determine_Cohort_files_pivot/Determine_lab_results_median_domain_expert_pivot.parquet")
else:
    lab_results_pivot_df.write_parquet("./Determine_Cohort_files_pivot/Determine_lab_results_median_all_features_pivot.parquet")

## Diagnoses pivot

In [257]:
if pivoting_config.diag_phemap_pivot:
    diagnoses_df = pl.read_parquet('./Determine_cohort_files/Determine_cohort_diagnoses_phecode_map_v2_with_z_codes.parquet') ### mention version of phecode
    diagnoses_df = diagnoses_df.with_columns(pl.lit(1).alias('Usage'))
    diagnoses_pivot_df = diagnoses_df.pivot(on = 'phecode_map', index = 'PATIENT_NUM', values = 'Usage', aggregate_function = 'max')
    diagnoses_pivot_df = diagnoses_pivot_df.fill_null(0)
    diagnoses_pivot_df.write_parquet('./Determine_Cohort_files_pivot/Determine_diagnoses_phemap_pivot_with_icd10z.parquet')
    
    
    diagnoses_df = pl.read_parquet('./Determine_cohort_files/Determine_cohort_diagnoses_phecode_map_v2_without_z_codes.parquet') ### mention version of phecode
    diagnoses_df = diagnoses_df.with_columns(pl.lit(1).alias('Usage'))
    diagnoses_pivot_df = diagnoses_df.pivot(on = 'phecode_map', index = 'PATIENT_NUM', values = 'Usage', aggregate_function = 'max')
    diagnoses_pivot_df = diagnoses_pivot_df.fill_null(0)
    diagnoses_pivot_df.write_parquet('./Determine_Cohort_files_pivot/Determine_diagnoses_phemap_pivot_without_icd10z.parquet')

In [259]:
if pivoting_config.diag_icd_nchars_pivot:
    diagnoses_df = pl.read_parquet('./Determine_cohort_files/Determine_cohort_diagnoses_initail_3_chars.parquet') ### mention version of phecode
    diagnoses_df = diagnoses_df.with_columns(pl.lit(1).alias('Usage'))
    diagnoses_pivot_df = diagnoses_df.pivot(on = 'get_first_3_char_icdcode', index = 'PATIENT_NUM', values = 'Usage', aggregate_function = 'max')
    diagnoses_pivot_df = diagnoses_pivot_df.fill_null(0)
    diagnoses_pivot_df.write_parquet('./Determine_Cohort_files_pivot/Determine_diagnoses_initial_3_chars_with_icd10z.parquet')

# CVS pivot

In [260]:
community_vital_pivot_df = community_vital_df.collect().pivot(on = 'Indicator', 
                                                    index ='PATIENT_NUM', 
                                                    values ='FACT', 
                                                    aggregate_function = 'mean')
community_vital_pivot_df = community_vital_pivot_df.fill_null(-100)

In [261]:
community_vital_pivot_df

PATIENT_NUM,ACS_GINI,ACS_MedHHIncome,ACS_pctCollGrad,ACS_pctPoverty100,ACS_Unemployment
i64,f32,f32,f32,f32,f32
5627352,41.849998,66088.0,34.7766,8.44147,2.68865
5621397,41.849998,66088.0,34.7766,8.44147,2.68865
563554,41.849998,66088.0,34.7766,8.44147,2.68865
528640,41.849998,66088.0,34.7766,8.44147,2.68865
517732,41.849998,66088.0,34.7766,8.44147,2.68865
…,…,…,…,…,…
6191930,42.75,74678.0,25.4405,8.82826,3.72293
509367,42.75,74678.0,25.4405,8.82826,3.72293
73920,42.75,74678.0,25.4405,8.82826,3.72293


In [262]:
community_vital_pivot_df.write_parquet('./Determine_modeling_datasets/Determine_cvs_pivot.parquet')

# Tobacco and smoking

In [ ]:
tob_pivot_df = tob_df.collect().pivot(on = 'CONCEPT_CD', 
                                                    index ='PATIENT_NUM', 
                                                    values ='Indicator', 
                                                    aggregate_function = 'first')

In [ ]:
tob_pivot_df.head()

In [ ]:
tob_pivot_df = tob_pivot_df.fill_null(0)